In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

ideas:

change in who a county votes for (flips or not) is spatially correlated / informed 

spatial correlation on residuals from a model that predicts winner in each county

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# load the full census election data geojson
census_election_data = gpd.read_file('../../data/county_demographics_with_elections_2012_2024.geojson')

In [ ]:
# write to a new geojson file
census_election_data.to_file('../../data/county_demographics_with_elections_2012_2024_resave.geojson', driver='GeoJSON')

In [ ]:
census_election_data.head()

##### Religion

In [ ]:
# open the relgion csv
religion_data = pd.read_csv('religion_groups_county.csv')

In [ ]:
religion_data

In [ ]:
religion_data.isna().sum()

In [ ]:
# drop rows with all NaN in Adherents	Adherents as % of Total Adherents	Adherents as % of Total Population
religion_data = religion_data.dropna(subset=['Adherents', 'Adherents as % of Total Adherents', 'Adherents as % of Total Population'], how='all')

In [ ]:
# drop Congregations column (weird NaN bug)
religion_data = religion_data.drop(columns=['Congregations'])

In [ ]:
# change all values in the religion data to be percentages instead of strings with percentage signs.  also renmove commas from numbers and convert to integers
# Adherents as % of Total Adherents, Adherents as % of Total Population, Adherents
religion_data['Adherents as % of Total Adherents'] = religion_data['Adherents as % of Total Adherents'].str.replace('%', '').str.replace(',', '').astype(float) / 100
religion_data['Adherents as % of Total Population'] = religion_data['Adherents as % of Total Population'].str.replace('%', '').str.replace(',', '').astype(float) / 100
religion_data['Adherents'] = religion_data['Adherents'].str.replace(',', '').astype(int)

In [ ]:
# show the top 5 group names with the most adherents
religion_data.groupby('Group Name')['Adherents'].sum().sort_values(ascending=False).head(20)

In [ ]:
religion_data.groupby('Group Name')['Adherents'].sum().sort_values(ascending=False).head(20).sum()

In [ ]:
religion_data['Group Name'].nunique()

In [ ]:
145119464/161009516

In [ ]:
# drop all rows where the group name is not in the top 20:
# 'Catholic Church', 'Non-denominational Christian Churches',
#        'Southern Baptist Convention', 'United Methodist Church',
#        'Church of Jesus Christ of Latter-day Saints', 'Muslim Estimate',
#        'Evangelical Lutheran Church in America', 'Assemblies of God',
#        'Jehovah's Witnesses', 'National Missionary Baptist Convention, Inc.',
#        'Lutheran Church--Missouri Synod', 'Episcopal Church',
#        'National Baptist Convention, USA, Inc.',
#        'Presbyterian Church (U.S.A.)', 'Churches of Christ',
#        'Christian Churches and Churches of Christ',
#        'Seventh-day Adventist Church', 'American Baptist Churches in the USA',
#        'African Methodist Episcopal Church', 'Orthodox Judaism'],
religion_data = religion_data[religion_data['Group Name'].isin([
    'Catholic Church', 'Non-denominational Christian Churches',
    'Southern Baptist Convention', 'United Methodist Church',
    'Church of Jesus Christ of Latter-day Saints', 'Muslim Estimate',
    'Evangelical Lutheran Church in America', 'Assemblies of God',
    "Jehovah's Witnesses", 'National Missionary Baptist Convention, Inc.',
    'Lutheran Church--Missouri Synod', 'Episcopal Church',
    'National Baptist Convention, USA, Inc.',
    'Presbyterian Church (U.S.A.)', 'Churches of Christ',
    'Christian Churches and Churches of Christ',
    'Seventh-day Adventist Church', 'American Baptist Churches in the USA',
    'African Methodist Episcopal Church', 'Orthodox Judaism'
])]

In [ ]:
len(religion_data['Group Name'].unique())

In [ ]:
import pandas as pd

def transform_religion_data(df):
    """
    Transform religion data from long to wide format with one row per county
    and separate columns for each religion's metrics.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with religion data in long format
    
    Returns:
    pandas.DataFrame: Transformed dataframe with one row per county
    """
    
    # Create base identifier columns for counties
    id_cols = ['FIPS', 'State Name', 'County Name']
    
    # Create three separate pivot tables for each metric
    adherents_pivot = df.pivot(
        index=id_cols,
        columns='Group Name',
        values='Adherents'
    ).add_suffix('_adherents')
    
    pct_adherents_pivot = df.pivot(
        index=id_cols,
        columns='Group Name',
        values='Adherents as % of Total Adherents'
    ).add_suffix('_percent_adherents_of_total_adherents')
    
    pct_pop_pivot = df.pivot(
        index=id_cols,
        columns='Group Name',
        values='Adherents as % of Total Population'
    ).add_suffix('_percent_adherents_of_total_population')
    
    # Combine all pivot tables
    result = pd.concat([
        adherents_pivot,
        pct_adherents_pivot,
        pct_pop_pivot
    ], axis=1)
    
    # Reset index to make FIPS, State Name, and County Name regular columns
    result = result.reset_index()
    
    # Clean up column names and add 'religion_' prefix to all religion-related columns
    result.columns = [
        ('religion_' + col.replace(' ', '_').replace("'", '').replace(',', '').replace('-', '_').lower()
         if col not in id_cols else col.replace(' ', '_').lower())
        for col in result.columns
    ]
    
    return result

In [ ]:
religion_transformed = transform_religion_data(religion_data)

In [ ]:
religion_transformed.fillna(0, inplace=True)

In [ ]:
# drop rows with state_name == 'District of Columbia', 'Alaska', 'Hawaii', 'Puerto Rico'
religion_transformed = religion_transformed[~religion_transformed['state_name'].isin(['District of Columbia', 'Alaska', 'Hawaii', 'Puerto Rico'])]

In [ ]:
# change fips to float
religion_transformed['fips'] = religion_transformed['fips'].astype(float)

In [ ]:
# merge religion_transformed with census_election_data on fips and FIPS Code,respectively
merged_data = pd.merge(census_election_data, religion_transformed, left_on='FIPS Code', right_on='fips', how='inner')

In [ ]:
merged_data

##### Features

In [ ]:
def add_race_percentages(df):
    """
    Add percentage columns for each race category with 'pct_' prefix.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing race population data
    
    Returns:
    pandas.DataFrame: Original DataFrame with new percentage columns added
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Define the years and categories
    years = [2012, 2016, 2020]
    categories = [
        'American Indian and Alaska Native alone',
        'Asian alone',
        'Black or African American alone',
        'Native Hawaiian and Other Pacific Islander alone',
        'Some other race alone',
        'Two or more races:',
        'White alone'
    ]
    
    # Calculate percentages for each year and category for each county
    # iterate over all rows
    for i, row in df.iterrows():
        # iterate over all years
        for year in years:
            # iterate over all categories
            for category in categories:
                # calculate the percentage
                total = row[f'Total:_{year}']
                value = row[f'{category}_{year}']
                pct = round((value / total) , 4)
                # add the percentage to the DataFrame
                df.at[i, f'pct_{category}_{year}'] = pct

    return df

df2 = add_race_percentages(census_election_data)

In [ ]:
# add a column called 'president_winner_{year}' that contains the party of the candidate that won the presidential election in that year

def add_president_winner(df):
    """
    Add a column containing the party of the candidate that won the presidential election in that year.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing election data
    
    Returns:
    pandas.DataFrame: Original DataFrame with new column added
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Define the years
    years = [2012, 2016, 2020]
    
    # Define the election winners
    winners = {
        2012: 'DEM',
        2016: 'GOP',
        2020: 'DEM'
    }
    
    # Add a column for each year containing the party of the election winner
    for year in years:
        df[f'president_winner_{year}'] = winners[year]
    
    return df

In [ ]:
df2 = add_president_winner(df2)

In [ ]:
census_election_data['president_winner_2024']

##### Split off data for election results and racial demographics

In [ ]:
def split_columns(df):

    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Define the columns to split off
    columns = [
        'geometry',
        'county_name',
        'state_name',
        'total_votes_2012',
        'votes_dem_2012',
        'votes_gop_2012',
        'per_dem_2012',
        'per_gop_2012',
        'diff_2012',
        'per_point_diff_2012',
        'total_votes_2016',
        'votes_dem_2016',
        'votes_gop_2016',
        'per_dem_2016',
        'per_gop_2016',
        'diff_2016',
        'per_point_diff_2016',
        'votes_gop_2020',
        'votes_dem_2020',
        'total_votes_2020',
        'diff_2020',
        'per_gop_2020',
        'per_dem_2020',
        'per_point_diff_2020',
        'winner_2012',
        'winner_2016',
        'winner_2020',
        'president_winner_2012',
        'president_winner_2016',
        'president_winner_2020',
        'Total:_2012',
        'pct_American Indian and Alaska Native alone_2012',
        'pct_Asian alone_2012',
        'pct_Black or African American alone_2012',
        'pct_Native Hawaiian and Other Pacific Islander alone_2012',
        'pct_Some other race alone_2012',
        'pct_Two or more races:_2012',
        'pct_White alone_2012',
        'Total:_2016',
        'pct_American Indian and Alaska Native alone_2016',
        'pct_Asian alone_2016',
        'pct_Black or African American alone_2016',
        'pct_Native Hawaiian and Other Pacific Islander alone_2016',
        'pct_Some other race alone_2016',
        'pct_Two or more races:_2016',
        'pct_White alone_2016',
        'Total:_2020',
        'pct_American Indian and Alaska Native alone_2020',
        'pct_Asian alone_2020',
        'pct_Black or African American alone_2020',
        'pct_Native Hawaiian and Other Pacific Islander alone_2020',
        'pct_Some other race alone_2020',
        'pct_Two or more races:_2020',
        'pct_White alone_2020',
        'FIPS Code'
    ]

    # Split off the columns
    df = df[columns]

    return df

In [ ]:
df_race_and_election = split_columns(df2)

##### Election voting analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

def analyze_and_visualize_elections(df):
    """
    One-line function to analyze and visualize county importance in presidential elections.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing county election data
    
    Returns:
    tuple: (analysis_results, fig) containing dictionary of analysis results and matplotlib figure
    """
    # Calculate metrics (reusing previous analysis logic)
    df['avg_total_votes'] = df[['total_votes_2012', 'total_votes_2016', 'total_votes_2020']].mean(axis=1)
    df['margin_shift_2012_2016'] = df['per_point_diff_2016'] - df['per_point_diff_2012']
    df['margin_shift_2016_2020'] = df['per_point_diff_2020'] - df['per_point_diff_2016']
    df['margin_volatility'] = np.abs(df['margin_shift_2012_2016']) + np.abs(df['margin_shift_2016_2020'])
    df['avg_margin'] = np.abs(df[['per_point_diff_2012', 'per_point_diff_2016', 'per_point_diff_2020']].mean(axis=1))
    df['party_changes'] = ((df['winner_2016'] != df['winner_2012']).astype(int) + 
                          (df['winner_2020'] != df['winner_2016']).astype(int))
    df['electoral_power_index'] = ((df['avg_total_votes'] / df['avg_total_votes'].max()) * 
                                 (1 - (np.abs(df['avg_margin']) / 100)))

    # Create visualization
    fig = plt.figure(figsize=(20, 15))
    
    # Subplot 1: Electoral Power Bubble Plot
    plt.subplot(2, 2, 1)
    plt.scatter(df['avg_margin'], np.log10(df['avg_total_votes']), 
               s=df['electoral_power_index']*1000, alpha=0.5)
    plt.xlabel('Average Margin (%)')
    plt.ylabel('Log10(Average Total Votes)')
    plt.title('County Electoral Power\n(Bubble size = Electoral Power Index)')
    
    # Add annotations for top 10 most important counties
    top_10 = df.nlargest(10, 'electoral_power_index')
    for _, county in top_10.iterrows():
        plt.annotate(f"{county['county_name']}, {county['state_name']}", 
                    (county['avg_margin'], np.log10(county['avg_total_votes'])))
    
    # Subplot 2: Swing County Analysis
    plt.subplot(2, 2, 2)
    swing_counties = df[df['party_changes'] > 0]
    sns.scatterplot(data=swing_counties, x='margin_volatility', y='avg_total_votes', 
                   hue='party_changes', size='avg_total_votes', sizes=(100, 1000))
    plt.title('Swing County Analysis')
    plt.xlabel('Margin Volatility')
    plt.ylabel('Average Total Votes')
    
    # Subplot 3: Demographic Change Impact
    plt.subplot(2, 2, 3)
    demographic_cols_2020 = [col for col in df.columns if col.startswith('pct_') and col.endswith('_2020')]
    demographic_cols_2012 = [col.replace('2020', '2012') for col in demographic_cols_2020]
    total_change = pd.DataFrame()
    for col_2012, col_2020 in zip(demographic_cols_2012, demographic_cols_2020):
        total_change[f'change_{col_2012}'] = df[col_2020] - df[col_2012]
    df['demographic_change_magnitude'] = np.abs(total_change).sum(axis=1)
    
    plt.scatter(df['demographic_change_magnitude'], df['margin_volatility'], 
               c=df['electoral_power_index'], cmap='viridis')
    plt.colorbar(label='Electoral Power Index')
    plt.xlabel('Magnitude of Demographic Change')
    plt.ylabel('Margin Volatility')
    plt.title('Demographic Change vs. Electoral Volatility')
    
    # Subplot 4: County Categories
    plt.subplot(2, 2, 4)
    competitive_threshold = 0.1
    df['category'] = 'Stable'
    df.loc[df['party_changes'] > 0, 'category'] = 'Swing'
    df.loc[(np.abs(df['avg_margin']) < competitive_threshold) & 
           (df['avg_total_votes'] > df['avg_total_votes'].quantile(0.75)), 'category'] = 'Battleground'
    df.loc[df['electoral_power_index'] > df['electoral_power_index'].quantile(0.95), 'category'] = 'High Impact'
    
    category_counts = df['category'].value_counts()
    plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%')
    plt.title('Distribution of County Categories')
    
    plt.tight_layout()
    
    # Compile analysis results
    results = {
        'high_impact_counties': df.nlargest(20, 'electoral_power_index')[
            ['county_name', 'state_name', 'electoral_power_index', 'avg_total_votes', 'avg_margin']
        ],
        'swing_counties': df[df['party_changes'] > 0].sort_values(
            by=['party_changes', 'avg_total_votes'], ascending=[False, False]
        )[['county_name', 'state_name', 'party_changes', 'margin_volatility']].head(20),
        'battleground_counties': df[
            (np.abs(df['avg_margin']) < competitive_threshold) &
            (df['avg_total_votes'] > df['avg_total_votes'].quantile(0.75))
        ][['county_name', 'state_name', 'avg_margin', 'avg_total_votes']].head(20),
        'demographic_shift_counties': df.nlargest(20, 'demographic_change_magnitude')[
            ['county_name', 'state_name', 'demographic_change_magnitude']
        ]
    }
    
    return results, fig

# Usage example:
# results, fig = analyze_and_visualize_elections(your_dataframe)
# fig.show()
# print(results)

##### Electoral analysis

In [ ]:
def add_electoral_votes(df):
    """
    Adds electoral vote counts for each state based on 2020/2024 electoral college allocation.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing a state_name column
    
    Returns:
    pandas.DataFrame: Original DataFrame with added electoral_votes column
    """
    
    electoral_votes_dict = {
        'Alabama': 9,
        'Alaska': 3,
        'Arizona': 11,
        'Arkansas': 6,
        'California': 54,
        'Colorado': 10,
        'Connecticut': 7,
        'Delaware': 3,
        'District of Columbia': 3,
        'Florida': 30,
        'Georgia': 16,
        'Hawaii': 4,
        'Idaho': 4,
        'Illinois': 19,
        'Indiana': 11,
        'Iowa': 6,
        'Kansas': 6,
        'Kentucky': 8,
        'Louisiana': 8,
        'Maine': 4,
        'Maryland': 10,
        'Massachusetts': 11,
        'Michigan': 15,
        'Minnesota': 10,
        'Mississippi': 6,
        'Missouri': 10,
        'Montana': 4,
        'Nebraska': 5,
        'Nevada': 6,
        'New Hampshire': 4,
        'New Jersey': 14,
        'New Mexico': 5,
        'New York': 28,
        'North Carolina': 16,
        'North Dakota': 3,
        'Ohio': 17,
        'Oklahoma': 7,
        'Oregon': 8,
        'Pennsylvania': 19,
        'Rhode Island': 4,
        'South Carolina': 9,
        'South Dakota': 3,
        'Tennessee': 11,
        'Texas': 40,
        'Utah': 6,
        'Vermont': 3,
        'Virginia': 13,
        'Washington': 12,
        'West Virginia': 4,
        'Wisconsin': 10,
        'Wyoming': 3
    }
    
    # Create a copy of the dataframe to avoid modifying the original
    df_with_ev = df.copy()
    
    # Add electoral votes column
    df_with_ev['electoral_votes'] = df_with_ev['state_name'].map(electoral_votes_dict)
    
    # Calculate state's share of electoral votes for each county
    df_with_ev['state_total_votes'] = df_with_ev.groupby('state_name')['avg_total_votes'].transform('sum')
    df_with_ev['county_vote_share'] = df_with_ev['avg_total_votes'] / df_with_ev['state_total_votes']
    df_with_ev['county_electoral_importance'] = df_with_ev['county_vote_share'] * df_with_ev['electoral_votes']
    
    # Add columns for electoral importance considering competitiveness
    df_with_ev['competitive_electoral_importance'] = (
        df_with_ev['county_electoral_importance'] * 
        (1 - abs(df_with_ev['avg_margin'])/100)  # Counties with closer margins get higher weight
    )
    
    return df_with_ev

def analyze_state_importance(df_with_ev):
    """
    Analyzes state-level electoral importance based on county data.
    
    Parameters:
    df_with_ev (pandas.DataFrame): DataFrame with electoral vote data added
    
    Returns:
    dict: Different metrics of state importance
    """
    state_analysis = {}
    
    # Aggregate to state level
    state_summary = df_with_ev.groupby('state_name').agg({
        'electoral_votes': 'first',  # Each state has same value for all counties
        'avg_margin': 'mean',
        'margin_volatility': 'mean',
        'avg_total_votes': 'sum',
        'party_changes': 'mean'
    }).reset_index()
    
    # Calculate state importance metrics
    state_summary['state_power_index'] = (
        (state_summary['electoral_votes'] / state_summary['electoral_votes'].max()) *  # Normalized electoral votes
        (1 - abs(state_summary['avg_margin'])/100) *  # Competitiveness factor
        (1 + state_summary['margin_volatility'])  # Volatility bonus
    )
    
    # Get different rankings
    state_analysis['most_powerful_states'] = state_summary.nlargest(10, 'state_power_index')
    state_analysis['most_competitive_states'] = state_summary.nlargest(10, 'margin_volatility')
    state_analysis['largest_electoral_votes'] = state_summary.nlargest(10, 'electoral_votes')
    
    return state_analysis

def visualize_state_importance(df_with_ev):
    """
    Creates visualizations of state electoral importance.
    
    Parameters:
    df_with_ev (pandas.DataFrame): DataFrame with electoral vote data added
    
    Returns:
    matplotlib.figure.Figure: Figure containing visualizations
    """
    fig = plt.figure(figsize=(20, 10))
    
    # State Power Index Plot
    state_summary = df_with_ev.groupby('state_name').agg({
        'electoral_votes': 'first',
        'avg_margin': 'mean',
        'margin_volatility': 'mean'
    }).reset_index()
    
    plt.subplot(1, 2, 1)
    plt.scatter(state_summary['avg_margin'], 
               state_summary['electoral_votes'],
               s=state_summary['margin_volatility']*100,
               alpha=0.6)
    
    # Label important states
    for idx, row in state_summary.nlargest(10, 'electoral_votes').iterrows():
        plt.annotate(row['state_name'], 
                    (row['avg_margin'], row['electoral_votes']))
    
    plt.xlabel('Average Margin (%)')
    plt.ylabel('Electoral Votes')
    plt.title('State Electoral Importance\n(Bubble size = Margin Volatility)')
    
    # Electoral Votes Distribution
    plt.subplot(1, 2, 2)
    state_summary.nlargest(15, 'electoral_votes').plot(
        kind='bar', 
        x='state_name', 
        y='electoral_votes',
        title='Top 15 States by Electoral Votes'
    )
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    return fig

In [ ]:
# Run analysis and get visualizations
results, fig = analyze_and_visualize_elections(df_race_and_election)

# Display the visualizations
plt.show()

# View the results
for category, data in results.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    print(data)

In [ ]:
# census_election_data move state_name, county_name to the first columns
def move_columns_to_front(df, columns):
    """
    Moves specified columns to the front of the DataFrame.
    
    Parameters:
    df (pandas.DataFrame): DataFrame to modify
    columns (list): List of column names to move to the front
    
    Returns:
    pandas.DataFrame: DataFrame with specified columns moved to the front
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Get list of columns not in the specified list
    other_columns = [col for col in df.columns if col not in columns]
    
    # Reorder columns
    new_columns = columns + other_columns
    df = df[new_columns]
    
    return df

census_election_data = move_columns_to_front(census_election_data, ['state_name', 'county_name'])

##### Vote Flipping

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights import Queen
from esda.moran import Moran
import folium
from matplotlib.colors import LinearSegmentedColormap

def analyze_spatial_vote_flipping(df):
    """
    Analyzes spatial correlation in county voting pattern changes.
    
    Parameters:
    df (GeoDataFrame): DataFrame containing county election data and geometries
    
    Returns:
    tuple: (analysis_results, figures) containing statistics and visualizations
    """
    # Convert to GeoDataFrame if not already
    gdf = gpd.GeoDataFrame(df) if not isinstance(df, gpd.GeoDataFrame) else df
    
    # Calculate flips between elections
    gdf['flip_2012_2016'] = (gdf['winner_2012'] != gdf['winner_2016']).astype(int)
    gdf['flip_2016_2020'] = (gdf['winner_2016'] != gdf['winner_2020']).astype(int)
    gdf['total_flips'] = gdf['flip_2012_2016'] + gdf['flip_2016_2020']
    
    # Create spatial weights matrix using Queen contiguity
    weights = Queen.from_dataframe(gdf)
    weights.transform = 'r'  # Row-standardize weights
    
    # Calculate Moran's I for different periods
    moran_2012_2016 = Moran(gdf['flip_2012_2016'], weights)
    moran_2016_2020 = Moran(gdf['flip_2016_2020'], weights)
    moran_total = Moran(gdf['total_flips'], weights)
    
    # Calculate local Moran's I for clustering analysis
    from esda.moran import Moran_Local
    local_moran = Moran_Local(gdf['total_flips'], weights)
    
    # Add cluster categories to the dataframe
    gdf['cluster_category'] = 'Not Significant'
    # High-High
    gdf.loc[(local_moran.p_sim < 0.05) & (gdf['total_flips'] > gdf['total_flips'].mean()) & 
            (local_moran.q == 1), 'cluster_category'] = 'High-High'
    # Low-Low
    gdf.loc[(local_moran.p_sim < 0.05) & (gdf['total_flips'] < gdf['total_flips'].mean()) & 
            (local_moran.q == 2), 'cluster_category'] = 'Low-Low'
    # High-Low
    gdf.loc[(local_moran.p_sim < 0.05) & (gdf['total_flips'] > gdf['total_flips'].mean()) & 
            (local_moran.q == 3), 'cluster_category'] = 'High-Low'
    # Low-High
    gdf.loc[(local_moran.p_sim < 0.05) & (gdf['total_flips'] < gdf['total_flips'].mean()) & 
            (local_moran.q == 4), 'cluster_category'] = 'Low-High'
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))
    
    # Plot 1: 2012-2016 flips
    gdf.plot(column='flip_2012_2016', 
            ax=axes[0,0],
            legend=True,
            legend_kwds={'label': 'County Flipped 2012-2016'},
            cmap='RdBu')
    axes[0,0].set_title("County Flips 2012-2016")
    axes[0,0].axis('off')
    
    # Plot 2: 2016-2020 flips
    gdf.plot(column='flip_2016_2020',
            ax=axes[0,1],
            legend=True,
            legend_kwds={'label': 'County Flipped 2016-2020'},
            cmap='RdBu')
    axes[0,1].set_title("County Flips 2016-2020")
    axes[0,1].axis('off')
    
    # Plot 3: Total flips
    gdf.plot(column='total_flips',
            ax=axes[1,0],
            legend=True,
            legend_kwds={'label': 'Total Flips'},
            cmap='viridis')
    axes[1,0].set_title("Total County Flips 2012-2020")
    axes[1,0].axis('off')
    
    # Plot 4: Cluster categories
    gdf.plot(column='cluster_category',
            ax=axes[1,1],
            legend=True,
            legend_kwds={'label': 'Spatial Clusters'},
            categorical=True,
            cmap='Set3')
    axes[1,1].set_title("Spatial Clustering of Flips")
    axes[1,1].axis('off')
    
    # Compile results
    results = {
        'global_spatial_correlation': {
            '2012-2016': {
                'morans_i': moran_2012_2016.I,
                'p_value': moran_2012_2016.p_sim
            },
            '2016-2020': {
                'morans_i': moran_2016_2020.I,
                'p_value': moran_2016_2020.p_sim
            },
            'total': {
                'morans_i': moran_total.I,
                'p_value': moran_total.p_sim
            }
        },
        'cluster_summary': gdf['cluster_category'].value_counts().to_dict(),
        'flip_summary': {
            'total_flipping_counties_2012_2016': gdf['flip_2012_2016'].sum(),
            'total_flipping_counties_2016_2020': gdf['flip_2016_2020'].sum(),
            'counties_that_flipped_twice': len(gdf[gdf['total_flips'] == 2])
        }
    }
    
    return results, fig, gdf

def print_spatial_analysis_results(results):
    """
    Prints formatted analysis results
    """
    print("Spatial Correlation Analysis of County Vote Flipping\n")
    
    print("Global Moran's I Statistics:")
    for period, stats in results['global_spatial_correlation'].items():
        print(f"\n{period}:")
        print(f"Moran's I: {stats['morans_i']:.3f}")
        print(f"P-value: {stats['p_value']:.3f}")
    
    print("\nCluster Analysis:")
    for cluster_type, count in results['cluster_summary'].items():
        print(f"{cluster_type}: {count} counties")
    
    print("\nFlip Summary:")
    for metric, value in results['flip_summary'].items():
        print(f"{metric.replace('_', ' ').title()}: {value}")

In [ ]:
# Run the analysis
results, fig, gdf_with_clusters = analyze_spatial_vote_flipping(df_race_and_election)

# Print the results
print_spatial_analysis_results(results)

# Show the maps
plt.show()

##### County election importance

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_county_importance(gdf, state_electoral_votes=None):
    """
    Analyzes county importance in presidential elections based on multiple factors.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame with county election data and geometries
    state_electoral_votes : dict, optional
        Dictionary of state electoral votes. If None, uses 2020 allocation.
        
    Returns:
    --------
    GeoDataFrame with importance metrics, Figure with visualizations
    """
    
    # Electoral votes by state (2020/2024 allocation)
    if state_electoral_votes is None:
        state_electoral_votes = {
            'Alabama': 9, 'Alaska': 3, 'Arizona': 11, 'Arkansas': 6, 'California': 54,
            'Colorado': 10, 'Connecticut': 7, 'Delaware': 3, 'District of Columbia': 3,
            'Florida': 30, 'Georgia': 16, 'Hawaii': 4, 'Idaho': 4, 'Illinois': 19,
            'Indiana': 11, 'Iowa': 6, 'Kansas': 6, 'Kentucky': 8, 'Louisiana': 8,
            'Maine': 4, 'Maryland': 10, 'Massachusetts': 11, 'Michigan': 15,
            'Minnesota': 10, 'Mississippi': 6, 'Missouri': 10, 'Montana': 4,
            'Nebraska': 5, 'Nevada': 6, 'New Hampshire': 4, 'New Jersey': 14,
            'New Mexico': 5, 'New York': 28, 'North Carolina': 16, 'North Dakota': 3,
            'Ohio': 17, 'Oklahoma': 7, 'Oregon': 8, 'Pennsylvania': 19,
            'Rhode Island': 4, 'South Carolina': 9, 'South Dakota': 3,
            'Tennessee': 11, 'Texas': 40, 'Utah': 6, 'Vermont': 3, 'Virginia': 13,
            'Washington': 12, 'West Virginia': 4, 'Wisconsin': 10, 'Wyoming': 3
        }
    
    # Create working copy
    gdf = gdf.copy()
    
    # Add electoral votes
    gdf['electoral_votes'] = gdf['state_name'].map(state_electoral_votes)
    
    # Calculate metrics
    
    # 1. Average margin across elections (closer = more important)
    gdf['avg_margin'] = np.abs(pd.concat([
        gdf['per_point_diff_2012'],
        gdf['per_point_diff_2016'],
        gdf['per_point_diff_2020']
    ], axis=1)).mean(axis=1)
    
    # 2. Margin volatility (more volatile = more important)
    gdf['margin_volatility'] = np.abs(
        gdf['per_point_diff_2016'] - gdf['per_point_diff_2012']
    ) + np.abs(
        gdf['per_point_diff_2020'] - gdf['per_point_diff_2016']
    )
    
    # 3. Average total votes (more votes = more important)
    gdf['avg_total_votes'] = pd.concat([
        gdf['total_votes_2012'],
        gdf['total_votes_2016'],
        gdf['total_votes_2020']
    ], axis=1).mean(axis=1)
    
    # 4. County's share of state's electoral votes
    gdf['state_total_votes'] = gdf.groupby('state_name')['avg_total_votes'].transform('sum')
    gdf['electoral_vote_share'] = (
        gdf['avg_total_votes'] / gdf['state_total_votes'] * gdf['electoral_votes']
    )
    
    # 5. Calculate swing state bonus
    gdf['swing_state_bonus'] = (
        (1 - np.abs(gdf['per_point_diff_2020'] / 100)) * 
        (gdf['margin_volatility'] / gdf['margin_volatility'].max())
    )
    
    # Calculate final importance score
    # Normalize components
    for col in ['avg_margin', 'margin_volatility', 'avg_total_votes', 'electoral_vote_share']:
        gdf[f'{col}_norm'] = (gdf[col] - gdf[col].min()) / (gdf[col].max() - gdf[col].min())
    
    gdf['importance_score'] = (
        (1 - gdf['avg_margin_norm']) * 0.3 +  # Close margins (30% weight)
        gdf['margin_volatility_norm'] * 0.2 +  # Volatility (20% weight)
        gdf['avg_total_votes_norm'] * 0.2 +    # Vote volume (20% weight)
        gdf['electoral_vote_share'] * 0.2 +    # Electoral importance (20% weight)
        gdf['swing_state_bonus'] * 0.1         # Swing state bonus (10% weight)
    )
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))
    
    # Plot 1: Overall Importance Score
    gdf.plot(
        column='importance_score',
        ax=axes[0,0],
        legend=True,
        cmap='viridis',
        legend_kwds={'label': 'County Importance Score'}
    )
    axes[0,0].set_title('County Electoral Importance')
    axes[0,0].axis('off')
    
    # Plot 2: Margin Volatility
    gdf.plot(
        column='margin_volatility',
        ax=axes[0,1],
        legend=True,
        cmap='RdYlBu',
        legend_kwds={'label': 'Margin Volatility'}
    )
    axes[0,1].set_title('County Vote Margin Volatility')
    axes[0,1].axis('off')
    
    # Plot 3: Electoral Vote Share
    gdf.plot(
        column='electoral_vote_share',
        ax=axes[1,0],
        legend=True,
        cmap='Reds',
        legend_kwds={'label': 'Electoral Vote Share'}
    )
    axes[1,0].set_title('County Share of Electoral Votes')
    axes[1,0].axis('off')
    
    # Plot 4: Swing State Bonus
    gdf.plot(
        column='swing_state_bonus',
        ax=axes[1,1],
        legend=True,
        cmap='coolwarm',
        legend_kwds={'label': 'Swing State Bonus'}
    )
    axes[1,1].set_title('Swing State Importance')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    
    # Get top counties
    top_counties = gdf.nlargest(20, 'importance_score')[
        ['county_name', 'state_name', 'importance_score', 
         'avg_margin', 'margin_volatility', 'electoral_vote_share']
    ]
    
    return gdf, fig, top_counties

def print_importance_analysis(top_counties):
    """
    Prints formatted analysis of most important counties
    """
    print("\nMost Important Counties for Electoral Victory:\n")
    
    for idx, county in top_counties.iterrows():
        print(f"{county['county_name']}, {county['state_name']}")
        print(f"  Importance Score: {county['importance_score']:.3f}")
        print(f"  Average Margin: {county['avg_margin']:.1f}%")
        print(f"  Margin Volatility: {county['margin_volatility']:.1f}")
        print(f"  Electoral Vote Share: {county['electoral_vote_share']:.3f}")
        print()

In [ ]:
# Run the analysis
gdf_with_importance, fig, top_counties = analyze_county_importance(df_race_and_election)

# Print the results
print_importance_analysis(top_counties)

# Show the maps
plt.show()

##### Gentriification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_gentrification(df):
    """
    Analyzes county-level gentrification using multiple indicators.
    
    Parameters:
    df: DataFrame containing county-level census and demographic data
    
    Returns:
    DataFrame with gentrification scores and component metrics
    """
    # Create working copy
    gdf = df.copy()
    
    # 1. Educational Change (2012-2020)
    gdf['edu_change_score'] = (
        (gdf["Percent of adults with a bachelor's degree or higher, 2018-22"] -
         gdf["Percent of adults with a bachelor's degree or higher, 2008-12"]) / 
        gdf["Percent of adults with a bachelor's degree or higher, 2008-12"]
    )
    
    # 2. Income Changes (using median household income)
    gdf['income_change_score'] = (
        (gdf['Median_Household_Income_2021'] -
         gdf['median_household_income_dollars_occupied_housing_units_2012']) /
        gdf['median_household_income_dollars_occupied_housing_units_2012']
    )
    
    # 3. Demographic Changes (2012-2020)
    gdf['demographic_change_score'] = (
        # Change in white population percentage
        np.abs(gdf['pct_White alone_2020'] - gdf['pct_White alone_2012']) +
        # Change in Black population percentage
        np.abs(gdf['pct_Black or African American alone_2020'] - gdf['pct_Black or African American alone_2012']) +
        # Change in Asian population percentage
        np.abs(gdf['pct_Asian alone_2020'] - gdf['pct_Asian alone_2012'])
    )
    
    # 4. Housing Cost Changes
    # Calculate change in housing costs using rent data
    gdf['housing_cost_change_score'] = (
        (gdf['median_dollars_occupied_housing_units_2016'] -  # Using 2016 as endpoint due to data availability
         gdf['median_dollars_occupied_housing_units_2012']) /
        gdf['median_dollars_occupied_housing_units_2012']
    )
    
    # 5. Employment Changes
    gdf['employment_change_score'] = (
        (gdf['Unemployment_rate_2022'] - gdf['Unemployment_rate_2012'])
    )
    
    # Calculate overall gentrification score
    # Normalize each component
    for col in ['edu_change_score', 'income_change_score', 'demographic_change_score', 
                'housing_cost_change_score', 'employment_change_score']:
        gdf[f'{col}_norm'] = (gdf[col] - gdf[col].min()) / (gdf[col].max() - gdf[col].min())
    
    # Weighted combination for final score
    gdf['gentrification_score'] = (
        gdf['edu_change_score_norm'] * 0.25 +              # Education change
        gdf['income_change_score_norm'] * 0.25 +           # Income change
        gdf['demographic_change_score_norm'] * 0.2 +       # Demographic change
        gdf['housing_cost_change_score_norm'] * 0.2 +      # Housing cost change
        gdf['employment_change_score_norm'] * 0.1          # Employment change
    )
    
    # Categorize gentrification levels
    gdf['gentrification_level'] = pd.qcut(
        gdf['gentrification_score'], 
        q=5, 
        labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High']
    )
    
    return gdf

def visualize_gentrification(gdf):
    """
    Creates visualizations of gentrification analysis.
    
    Parameters:
    gdf: GeoDataFrame with gentrification metrics
    
    Returns:
    matplotlib figure
    """
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))
    
    # Plot 1: Overall Gentrification Score
    gdf.plot(
        column='gentrification_score',
        ax=axes[0,0],
        legend=True,
        cmap='RdYlBu_r',
        legend_kwds={'label': 'Gentrification Score'}
    )
    axes[0,0].set_title('Overall Gentrification Score')
    axes[0,0].axis('off')
    
    # Plot 2: Education and Income Changes
    ax2 = axes[0,1].scatter(
        gdf['edu_change_score'],
        gdf['income_change_score'],
        c=gdf['gentrification_score'],
        cmap='RdYlBu_r',
        alpha=0.6
    )
    axes[0,1].set_xlabel('Educational Attainment Change')
    axes[0,1].set_ylabel('Income Change')
    axes[0,1].set_title('Education vs Income Changes')
    plt.colorbar(ax2, ax=axes[0,1], label='Gentrification Score')
    
    # Plot 3: Housing Cost Change
    gdf.plot(
        column='housing_cost_change_score',
        ax=axes[1,0],
        legend=True,
        cmap='Reds',
        legend_kwds={'label': 'Housing Cost Change'}
    )
    axes[1,0].set_title('Housing Cost Changes')
    axes[1,0].axis('off')
    
    # Plot 4: Demographic Change
    gdf.plot(
        column='demographic_change_score',
        ax=axes[1,1],
        legend=True,
        cmap='viridis',
        legend_kwds={'label': 'Demographic Change Score'}
    )
    axes[1,1].set_title('Demographic Changes')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    return fig

def print_gentrification_summary(gdf):
    """
    Prints summary statistics and identifies most gentrified counties.
    
    Parameters:
    gdf: GeoDataFrame with gentrification metrics
    """
    print("Gentrification Analysis Summary\n")
    
    # Overall statistics
    print("Overall Statistics:")
    print(f"Mean Gentrification Score: {gdf['gentrification_score'].mean():.3f}")
    print(f"Median Gentrification Score: {gdf['gentrification_score'].median():.3f}")
    
    print("\nDistribution by Gentrification Level:")
    print(gdf['gentrification_level'].value_counts())
    
    print("\nTop 10 Most Gentrified Counties:")
    top_10 = gdf.nlargest(10, 'gentrification_score')[
        ['county_name', 'state_name', 'gentrification_score', 
         'edu_change_score', 'income_change_score', 'housing_cost_change_score']
    ]
    for idx, row in top_10.iterrows():
        print(f"\n{row['county_name']}, {row['state_name']}")
        print(f"  Gentrification Score: {row['gentrification_score']:.3f}")
        print(f"  Education Change: {row['edu_change_score']:.1%}")
        print(f"  Income Change: {row['income_change_score']:.1%}")
        print(f"  Housing Cost Change: {row['housing_cost_change_score']:.1%}")

In [ ]:
# Run analysis
gdf_with_gentrification = analyze_gentrification(df2)

# Create visualizations
fig = visualize_gentrification(gdf_with_gentrification)

# Print summary statistics
print_gentrification_summary(gdf_with_gentrification)

# Show plots
plt.show()

##### Industry

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_industry_occupation_changes(df):
    """
    Analyzes changes in occupational composition within industries across 2012, 2016, and 2020.
    
    Parameters:
    df: DataFrame containing industry and occupation data
    
    Returns:
    Tuple of (changes_df, summary_stats)
    """
    # Define main industries (excluding total employment)
    industries = {
        'agriculture': 'agriculture_forestry_fishing_and_hunting_and_mining',
        'arts': 'arts_entertainment_and_recreation_and_accommodation_and_food_services',
        'construction': 'construction',
        'education': 'educational_services_and_health_care_and_social_assistance',
        'finance': 'finance_and_insurance_and_real_estate_and_rental_and_leasing',
        'information': 'information',
        'manufacturing': 'manufacturing',
        'other_services': 'other_services_except_public_administration',
        'professional': 'professional_scientific_and_management_and_administrative_and_waste_management_services',
        'public_admin': 'public_administration',
        'retail': 'retail_trade',
        'transportation': 'transportation_and_warehousing_and_utilities',
        'wholesale': 'wholesale_trade'
    }
    
    # Define occupations
    occupations = {
        'management': 'management_business_science_and_arts_occupations',
        'service': 'service_occupations',
        'sales': 'sales_and_office_occupations',
        'construction': 'natural_resources_construction_and_maintenance_occupations',
        'production': 'production_transportation_and_material_moving_occupations'
    }
    
    # Initialize dictionaries to store DataFrames for concatenation
    base_values = {}
    changes_12_16 = {}
    changes_16_20 = {}
    changes_12_20 = {}
    pct_changes_12_16 = {}
    pct_changes_16_20 = {}
    pct_changes_12_20 = {}
    
    # Calculate all values and changes
    for ind_key, industry in industries.items():
        for occ_key, occupation in occupations.items():
            # Column pattern for this industry-occupation combination
            col_pattern = f"INDUSTRY_{industry}_OCCUPATION_{occupation}_percent"
            base_name = f'{ind_key}_{occ_key}'
            
            # Get values for each year
            values = {}
            for year in [2012, 2016, 2020]:
                col_name = f"{col_pattern}_{year}"
                if col_name in df.columns:
                    values[year] = df[col_name]
                    base_values[f'{base_name}_{year}'] = values[year]
            
            # Calculate changes if we have all years
            if len(values) == 3:
                changes_12_16[base_name] = values[2016] - values[2012]
                changes_16_20[base_name] = values[2020] - values[2016]
                changes_12_20[base_name] = values[2020] - values[2012]
                
                # Calculate percentage changes
                pct_changes_12_16[base_name] = ((values[2016] - values[2012]) / values[2012] * 100).replace([np.inf, -np.inf], np.nan)
                pct_changes_16_20[base_name] = ((values[2020] - values[2016]) / values[2016] * 100).replace([np.inf, -np.inf], np.nan)
                pct_changes_12_20[base_name] = ((values[2020] - values[2012]) / values[2012] * 100).replace([np.inf, -np.inf], np.nan)
    
    # Combine all DataFrames
    changes = pd.DataFrame({
        'county_name': df['county_name'],
        'state_name': df['state_name']
    })
    
    # Concatenate all the calculated values
    if base_values:
        changes = pd.concat([changes, pd.DataFrame(base_values)], axis=1)
    
    # Add the changes columns with appropriate suffixes
    for base_name in changes_12_16.keys():
        changes = pd.concat([
            changes,
            pd.DataFrame({
                f'{base_name}_change_12_16': changes_12_16[base_name],
                f'{base_name}_change_16_20': changes_16_20[base_name],
                f'{base_name}_change_12_20': changes_12_20[base_name],
                f'{base_name}_pct_change_12_16': pct_changes_12_16[base_name],
                f'{base_name}_pct_change_16_20': pct_changes_16_20[base_name],
                f'{base_name}_pct_change_12_20': pct_changes_12_20[base_name]
            })
        ], axis=1)
    
    # Calculate summary statistics
    summary_stats = {
        'industry_changes': {},
        'top_shifts': {},
        'volatility': {}
    }
    
    for ind_key, industry in industries.items():
        industry_stats = {}
        for occ_key, occupation in occupations.items():
            base_name = f'{ind_key}_{occ_key}'
            
            if f'{base_name}_change_12_20' in changes.columns:
                # Overall changes
                industry_stats[occ_key] = {
                    '2012-2016': changes[f'{base_name}_change_12_16'].mean(),
                    '2016-2020': changes[f'{base_name}_change_16_20'].mean(),
                    '2012-2020': changes[f'{base_name}_change_12_20'].mean()
                }
                
                # Calculate volatility
                summary_stats['volatility'][base_name] = changes[[
                    f'{base_name}_change_12_16',
                    f'{base_name}_change_16_20'
                ]].std().mean()
                
                # Top shifts
                summary_stats['top_shifts'][base_name] = {
                    'increasing': changes.nlargest(
                        5, f'{base_name}_change_12_20'
                    )[['county_name', 'state_name', f'{base_name}_change_12_20']],
                    'decreasing': changes.nsmallest(
                        5, f'{base_name}_change_12_20'
                    )[['county_name', 'state_name', f'{base_name}_change_12_20']]
                }
        
        summary_stats['industry_changes'][ind_key] = industry_stats
    
    return changes, summary_stats

def print_industry_summary(summary_stats, industry_name):
    """
    Prints formatted summary of occupational changes within an industry.
    
    Parameters:
    summary_stats: Dictionary containing summary statistics
    industry_name: Key of the industry to summarize
    """
    print(f"\nOccupational Changes in {industry_name.title()} Industry\n")
    
    if industry_name in summary_stats['industry_changes']:
        industry_stats = summary_stats['industry_changes'][industry_name]
        
        print("Overall Changes (Percentage Points):")
        for occupation, changes in industry_stats.items():
            print(f"\n{occupation.title()}:")
            print(f"  2012-2016: {changes['2012-2016']:.3f}")
            print(f"  2016-2020: {changes['2016-2020']:.3f}")
            print(f"  2012-2020: {changes['2012-2020']:.3f}")
        
        print("\nMost Volatile Occupation Combinations:")
        relevant_volatility = {k: v for k, v in summary_stats['volatility'].items() 
                             if k.startswith(industry_name)}
        sorted_volatility = sorted(
            relevant_volatility.items(),
            key=lambda x: x[1],
            reverse=True
        )
        for combo, volatility in sorted_volatility:
            print(f"{combo.replace(f'{industry_name}_', '')}: {volatility:.3f}")

def visualize_industry_occupation_changes(changes_df, industry_name):
    """
    Creates visualizations of occupational changes within a specific industry.
    
    Parameters:
    changes_df: DataFrame containing calculated changes
    industry_name: Key of the industry to visualize
    
    Returns:
    matplotlib figure
    """
    occupations = ['management', 'service', 'sales', 'construction', 'production']
    
    fig = plt.figure(figsize=(20, 15))
    
    # Plot 1: Box plot of percentage changes with outlier handling
    plt.subplot(2, 2, 1)
    change_data = []
    labels = []
    
    # Define reasonable bounds for percentage changes
    lower_bound = -100  # Can't decrease more than 100%
    upper_bound = 200   # Limit increases to 200%
    
    for occ in occupations:
        base_name = f'{industry_name}_{occ}'
        if f'{base_name}_pct_change_12_16' in changes_df.columns:
            # Clip the percentage changes to reasonable bounds
            data_12_16 = changes_df[f'{base_name}_pct_change_12_16'].clip(lower_bound, upper_bound).dropna()
            data_16_20 = changes_df[f'{base_name}_pct_change_16_20'].clip(lower_bound, upper_bound).dropna()
            
            change_data.extend([data_12_16, data_16_20])
            labels.extend([f'{occ}\n2012-2016', f'{occ}\n2016-2020'])
    
    plt.boxplot(change_data, labels=labels, whis=1.5)  # Use 1.5 IQR for whiskers
    plt.xticks(rotation=45)
    plt.title(f'Distribution of Percentage Changes by Occupation\nin {industry_name.title()} Industry')
    plt.ylabel('Percentage Change')
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.2)  # Add reference line at 0
    
    # Add note about bounds
    plt.text(0.02, 0.98, f'Note: Changes clipped to [{lower_bound}%, {upper_bound}%]', 
             transform=plt.gca().transAxes, fontsize=8, verticalalignment='top')
    
    # Plot 2: Average composition over time
    plt.subplot(2, 2, 2)
    years = [2012, 2016, 2020]
    for occ in occupations:
        base_name = f'{industry_name}_{occ}'
        if all(f'{base_name}_{year}' in changes_df.columns for year in years):
            values = [
                changes_df[f'{base_name}_{year}'].mean() for year in years
            ]
            plt.plot(years, values, marker='o', label=occ)
    
    plt.legend()
    plt.title(f'Average Occupation Percentages Over Time\nin {industry_name.title()} Industry')
    plt.xlabel('Year')
    plt.ylabel('Percentage')
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Heatmap of correlations between occupation changes
    plt.subplot(2, 2, 3)
    
    # Use absolute changes instead of percentage changes for correlation
    change_cols = [f'{industry_name}_{occ}_change_12_20' for occ in occupations 
                  if f'{industry_name}_{occ}_change_12_20' in changes_df.columns]
    if change_cols:
        correlation_matrix = changes_df[change_cols].corr()
        
        sns.heatmap(
            correlation_matrix,
            annot=True,
            cmap='RdBu',
            center=0,
            vmin=-1,
            vmax=1,
            fmt='.2f'
        )
        plt.title(f'Correlation of Changes Between Occupations\nin {industry_name.title()} Industry')
        
        # Clean up axis labels
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        labels = [col.replace(f'{industry_name}_', '').replace('_change_12_20', '') for col in change_cols]
        plt.xticks(np.arange(len(labels)) + 0.5, labels)
        plt.yticks(np.arange(len(labels)) + 0.5, labels)
    
    # Add plot 4: Distribution of raw percentages in 2020
    plt.subplot(2, 2, 4)
    current_data = []
    current_labels = []
    
    for occ in occupations:
        base_name = f'{industry_name}_{occ}'
        if f'{base_name}_2020' in changes_df.columns:
            current_data.append(changes_df[f'{base_name}_2020'].dropna())
            current_labels.append(occ)
    
    if current_data:
        plt.boxplot(current_data, labels=current_labels)
        plt.xticks(rotation=45)
        plt.title(f'Distribution of Occupation Percentages (2020)\nin {industry_name.title()} Industry')
        plt.ylabel('Percentage')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

In [ ]:
# Run the analysis
changes, summary = analyze_industry_occupation_changes(census_election_data)

# Analyze a specific industry (e.g., manufacturing)
fig = visualize_industry_occupation_changes(changes, 'manufacturing')
print_industry_summary(summary, 'manufacturing')

# You can analyze any industry using its key:
# 'agriculture', 'arts', 'construction', 'education', 'finance', 
# 'information', 'manufacturing', 'other_services', 'professional', 
# 'public_admin', 'retail', 'transportation', 'wholesale'

#### Other stuff

In [ ]:
columns_to_select = [
    'geometry', 'county_name', 'state_name',
    
    # Election data 2012
    'total_votes_2012', 'votes_dem_2012', 'votes_gop_2012',
    'per_dem_2012', 'per_gop_2012', 'diff_2012', 'per_point_diff_2012',
    
    # Election data 2016
    'total_votes_2016', 'votes_dem_2016', 'votes_gop_2016',
    'per_dem_2016', 'per_gop_2016', 'diff_2016', 'per_point_diff_2016',
    
    # Election data 2020
    'votes_gop_2020', 'votes_dem_2020', 'total_votes_2020',
    'diff_2020', 'per_gop_2020', 'per_dem_2020', 'per_point_diff_2020',
    
    # Winners
    'winner_2012', 'winner_2016', 'winner_2020',
    
    # Race data 2012
    'Total:_2012',
    'pct_American Indian and Alaska Native alone_2012',
    'pct_Asian alone_2012',
    'pct_Black or African American alone_2012',
    'pct_Native Hawaiian and Other Pacific Islander alone_2012',
    'pct_Some other race alone_2012',
    'pct_Two or more races:_2012',
    'pct_White alone_2012',
    
    # Race data 2016
    'Total:_2016',
    'pct_American Indian and Alaska Native alone_2016',
    'pct_Asian alone_2016',
    'pct_Black or African American alone_2016',
    'pct_Native Hawaiian and Other Pacific Islander alone_2016',
    'pct_Some other race alone_2016',
    'pct_Two or more races:_2016',
    'pct_White alone_2016',
    
    # Race data 2020
    'Total:_2020',
    'pct_American Indian and Alaska Native alone_2020',
    'pct_Asian alone_2020',
    'pct_Black or African American alone_2020',
    'pct_Native Hawaiian and Other Pacific Islander alone_2020',
    'pct_Some other race alone_2020',
    'pct_Two or more races:_2020',
    'pct_White alone_2020',
]

subset_df = df2[columns_to_select]

In [ ]:
subset_df

In [ ]:
# change the per_point_dff_2016 column values from strings with percent signs to floats
subset_df['per_point_diff_2016'] = subset_df['per_point_diff_2016'].str.replace('%', '').astype(float)

In [ ]:
# plot the subsset_df , color by a value
subset_df.plot(column='pct_White alone_2012', legend=True, figsize=(15, 15), cmap='Blues')
plt.show()

In [ ]:
# save all col names to a file
with open('data/other/all_col_names.txt', 'w') as f:
    for col in census_election_data.columns:
        f.write(f'{col}\n')

In [ ]:
# show columns that have VALUE in the name
value = 'Metro'
total_columns = [col for col in census_election_data.columns if value in col]
print(total_columns)

In [ ]:
# show count of metro 2013 vals
print(census_election_data['Metro_2013'].value_counts())

In [ ]:
# show the counties with the largest changes in per_gop_2012 to per_gop_2016 (already columns)
census_election_data['gop_change'] = census_election_data['per_gop_2016'] - census_election_data['per_gop_2012']
census_election_data['gop_change_abs'] = abs(census_election_data['gop_change'])
census_election_data = census_election_data.sort_values('gop_change_abs', ascending=False)
print(census_election_data[['county_name', 'gop_change', 'gop_change_abs']].head(10))

In [ ]:
# plot counties by Metro_2013 (0=non-metro, 1=metro)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
census_election_data.plot(column='Metro_2013', ax=ax, legend=True)
plt.title('Metro_2013')
plt.show()

#### ML / Stats

In [ ]:
census_election_data.dtypes.value_counts()

# show all column names that are object type
object_columns = census_election_data.select_dtypes(include='object').columns
print(object_columns)

##### Correlations

In [ ]:
# # calculate correlation between all columns.  this should be returned in the form of a list with the top 100 highest ocrrelating pairs
# correlation_matrix = census_election_data.corr().abs()
# correlation_values = correlation_matrix.unstack().sort_values(ascending=False)
# correlation_values = correlation_values[correlation_values < 1]
# correlation_values = correlation_values[0:100]
# print(correlation_values)

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

def analyze_correlations(df, top_n=20, min_correlation=0.0):
    """
    Analyze correlations between all numeric columns in a dataframe with formatted output.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe
    top_n (int): Number of top correlations to return
    min_correlation (float): Minimum absolute correlation value to consider
    
    Returns:
    pandas.DataFrame: Top correlations sorted by absolute value
    """
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculate correlation matrix
    corr_matrix = numeric_df.corr()
    
    # Convert correlation matrix to long format
    correlations = []
    for col1 in corr_matrix.columns:
        for col2 in corr_matrix.columns:
            if col1 < col2:  # Only take upper triangle to avoid duplicates
                correlation = corr_matrix.loc[col1, col2]
                if abs(correlation) >= min_correlation:
                    correlations.append({
                        'Variable 1': col1,
                        'Variable 2': col2,
                        'Correlation': correlation,
                        'Absolute Correlation': abs(correlation)
                    })
    
    # Convert to dataframe and sort
    corr_df = pd.DataFrame(correlations)
    corr_df = corr_df.sort_values('Absolute Correlation', ascending=False)
    
    # Format correlation values
    corr_df['Correlation'] = corr_df['Correlation'].round(3)
    
    # Create formatted output
    # result = corr_df.head(top_n)[['Variable 1', 'Variable 2', 'Correlation']]
    result = corr_df[['Variable 1', 'Variable 2', 'Correlation']]
    
    # # Print formatted results
    # print("\nTop", top_n, "Correlations:")
    # print("=" * 100)
    # for idx, row in result.iterrows(): 
    #     print(f"\nCorrelation: {row['Correlation']:.3f}")
    #     print(f"Variable 1: {row['Variable 1']}")
    #     print(f"Variable 2: {row['Variable 2']}")
    #     print("-" * 100)
    
    return result

# Example usage:
# top_correlations = analyze_correlations(df, top_n=20)

In [ ]:
top_correlations = analyze_correlations(census_election_data, top_n=5000)

In [ ]:
# # drop all perfect correlations (1 or -1)
# top_correlations = top_correlations[top_correlations['Correlation'] < 1]
# top_correlations = top_correlations[top_correlations['Correlation'] > -1]

In [ ]:
# function to filter the top_correlations dataframe by a string in either Variable 1 or Variable 2 (separately), as well as a correlation value range
def filter_correlations(correlations, filter_str=None, min_correlation=None, max_correlation=None):
    """
    Filter correlation dataframe by variable names and correlation values.
    
    Parameters:
    correlations (pandas.DataFrame): DataFrame with correlation values
    filter_str (str): String to filter variable names
    min_correlation (float): Minimum correlation value
    max_correlation (float): Maximum correlation value
    
    Returns:
    pandas.DataFrame: Filtered correlation dataframe
    """
    # Filter by variable names
    if filter_str:
        correlations = correlations[
            correlations['Variable 1'].str.contains(filter_str) | 
            correlations['Variable 2'].str.contains(filter_str)
        ]
    
    # Filter by correlation values
    if min_correlation:
        correlations = correlations[correlations['Correlation'] >= min_correlation]
    if max_correlation:
        correlations = correlations[correlations['Correlation'] <= max_correlation]
    
    return correlations

In [ ]:
var_filter = 'religion'
min_corr = 0.7
max_corr = 1
filtered_correlations = filter_correlations(top_correlations, filter_str=var_filter, min_correlation=min_corr, max_correlation=max_corr)
# filtered_correlations = filter_correlations(top_correlations, min_correlation=min_corr, max_correlation=max_corr)
filtered_correlations

##### Model importance

In [ ]:
county_dataframe = pd.DataFrame(census_election_data.drop(columns=['geometry']))

In [ ]:
county_dataframe

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

class FeatureImportanceAnalyzer:
    def __init__(self):
        self.model = None
        self.feature_importances = None
        self.selected_features = None
        self.target = None
        self.scaler = StandardScaler()
        
    def _preprocess_data(self, df, columns_to_drop=None):
        """
        Preprocess the data by handling missing values and non-numeric columns.
        
        Parameters:
        -----------
        df : pandas.DataFrame
            Input dataframe
        columns_to_drop : list
            List of strings - columns containing any of these strings will be dropped
        """
        # Create a copy of the dataframe
        df_processed = df.copy()
        
        # Drop columns containing any of the strings in columns_to_drop
        if columns_to_drop:
            # Get all columns that contain any of the strings to drop
            cols_to_drop = []
            for pattern in columns_to_drop:
                matching_cols = df_processed.columns[df_processed.columns.str.contains(pattern, case=False)]
                cols_to_drop.extend(matching_cols)
            
            # Remove duplicates and ensure we don't drop the target
            cols_to_drop = list(set([col for col in cols_to_drop if col != self.target]))
            
            # Drop the columns
            df_processed = df_processed.drop(columns=cols_to_drop, errors='ignore')
            
            print(f"Dropped {len(cols_to_drop)} columns containing the patterns: {columns_to_drop}")
        
        # Handle missing values
        for column in df_processed.columns:
            if df_processed[column].dtype in ['int64', 'float64']:
                df_processed[column].fillna(df_processed[column].mean(), inplace=True)
            else:
                df_processed[column].fillna(df_processed[column].mode()[0], inplace=True)
        
        # Get numeric columns only, making sure to include the target column
        numeric_cols = df_processed.select_dtypes(include=['int64', 'float64']).columns
        if self.target not in numeric_cols:
            numeric_cols = numeric_cols.append(pd.Index([self.target]))
        
        return df_processed[numeric_cols]

    def analyze_importance(self, df, target_column, columns_to_drop=None, n_features=20):
        """
        Analyze feature importance for a given target column.
        
        Parameters:
        -----------
        df : pandas.DataFrame
            Input dataframe
        target_column : str
            Name of the target column
        columns_to_drop : list
            List of strings - columns containing any of these strings will be dropped
        n_features : int, optional
            Number of top features to return (default=20)
            
        Returns:
        --------
        dict : Dictionary containing:
            - feature_importances: DataFrame with feature importances
            - model_performance: Dict with R² and RMSE scores
            - model: Trained RandomForestRegressor
        """
        # Store target
        self.target = target_column
        
        # Preprocess data
        df_processed = self._preprocess_data(df, columns_to_drop)
        
        # Verify target column is present
        if target_column not in df_processed.columns:
            raise ValueError(f"Target column '{target_column}' not found in processed data")
        
        # Separate features and target
        X = df_processed.drop(columns=[target_column])
        y = df_processed[target_column]
        
        # Store selected features
        self.selected_features = X.columns.tolist()
        
        # Print number of features being used
        print(f"Using {len(self.selected_features)} features for analysis")
        
        # Split the data
        print("Splitting data...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale the features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        print("Training model...")
        self.model = RandomForestRegressor(n_estimators=100, random_state=42, verbose=1, n_jobs=-1)
        self.model.fit(X_train_scaled, y_train)
        
        # Get predictions
        y_pred = self.model.predict(X_test_scaled)
        
        # Calculate performance metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Get feature importances
        importances = self.model.feature_importances_
        self.feature_importances = pd.DataFrame({
            'feature': self.selected_features,
            'importance': importances
        }).sort_values('importance', ascending=False).head(n_features)
        
        return {
            'feature_importances': self.feature_importances,
            'model_performance': {
                'r2_score': r2,
                'rmse': rmse
            },
            'model': self.model
        }
    
    def plot_importance(self, n_features=20):
        """Plot feature importance using matplotlib."""
        try:
            import matplotlib.pyplot as plt
            
            plt.figure(figsize=(12, 8))
            plt.barh(self.feature_importances['feature'][:n_features],
                    self.feature_importances['importance'][:n_features])
            plt.xlabel('Feature Importance')
            plt.title(f'Top {n_features} Most Important Features for {self.target}')
            plt.gca().invert_yaxis()
            plt.tight_layout()
            plt.show()
        except ImportError:
            print("matplotlib is required for plotting. Please install it first.")

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
# Create analyzer instance
analyzer = FeatureImportanceAnalyzer()

# Define patterns to drop - any column containing these strings will be dropped
columns_to_drop = ['vote', 'winner', 'per']

# Analyze importance for a target variable
results = analyzer.analyze_importance(
    df=county_dataframe,
    target_column='winner_2016_numeric',
    columns_to_drop=columns_to_drop
)

# Print results
print("\nTop 10 Most Important Features:")
print(results['feature_importances'].head(10))

print("\nModel Performance:")
print(f"R² Score: {results['model_performance']['r2_score']:.4f}")
print(f"RMSE: {results['model_performance']['rmse']:.4f}")

# Plot results
analyzer.plot_importance(n_features=15)

In [ ]:
# show columns with religion in the name
religion_columns = [col for col in county_dataframe.columns if 'muslim' in col]
print(religion_columns)

In [ ]:
# show row where religion_jehovahs_witnesses_percent_adherents_of_total_population is max
max_row = county_dataframe[county_dataframe['religion_jehovahs_witnesses_percent_adherents_of_total_population'] == county_dataframe['religion_jehovahs_witnesses_percent_adherents_of_total_population'].max()]
max_row.county_name

In [ ]:
max_row['Total:_2020']

#### Maps

##### Simple Choropleth

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

def plot_choropleth(
    gdf,
    feature_name,
    scale='linear',
    figsize=(15, 10),
    cmap='YlOrRd',
    title=None,
    legend_label=None
):
    """
    Create a choropleth map from a GeoDataFrame with flexible scaling options.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        The geodataframe containing geographic and feature data
    feature_name : str
        The column name of the feature to plot
    scale : str, default 'linear'
        The scale type ('linear' or 'log')
    figsize : tuple, default (15, 10)
        Figure size as (width, height)
    cmap : str, default 'YlOrRd'
        Matplotlib colormap name
    title : str, optional
        Custom title for the plot. If None, will generate based on feature name
    legend_label : str, optional
        Custom label for the legend. If None, will use feature name
        
    Returns:
    --------
    fig, ax : tuple
        The figure and axis objects for further customization if needed
    """
    # Input validation
    if feature_name not in gdf.columns:
        raise ValueError(f"Feature '{feature_name}' not found in the GeoDataFrame")
    
    if scale not in ['linear', 'log']:
        raise ValueError("Scale must be either 'linear' or 'log'")
    
    # Create figure and axis
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Prepare the normalization
    if scale == 'log':
        # Handle zero or negative values for log scale
        min_val = gdf[feature_name].min()
        if min_val <= 0:
            print(f"Warning: Data contains values ≤ 0. Adding offset for log scale.")
            gdf = gdf.copy()
            gdf[feature_name] = gdf[feature_name] - min_val + 1
        
        norm = LogNorm(
            vmin=gdf[feature_name].min(),
            vmax=gdf[feature_name].max()
        )
    else:
        norm = None
    
    # Generate default title and legend label if not provided
    if title is None:
        title = f"{feature_name} by County"
        if scale == 'log':
            title += " (Log Scale)"
            
    if legend_label is None:
        legend_label = feature_name
        if scale == 'log':
            legend_label += " (log scale)"
    
    # Create the plot
    gdf.plot(
        column=feature_name,
        ax=ax,
        legend=True,
        legend_kwds={
            'label': legend_label,
            'orientation': 'vertical'
        },
        cmap=cmap,
        norm=norm
    )
    
    # Customize the appearance
    plt.title(title, pad=20)
    ax.axis('off')
    plt.tight_layout()
    
    return fig, ax

In [ ]:
plot_choropleth(census_election_data, 'winner_2024_numeric', scale='linear', cmap='viridis')

##### Multi feature Choropleth (folium)

In [ ]:
import folium
import numpy as np
from branca.colormap import LinearColormap
import pandas as pd

# Define color schemes
COLOR_SCHEMES = {
    'population': ['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15'],  # Reds
    'political': ['#0571b0', '#92c5de', '#f7f7f7', '#f4a582', '#ca0020'],   # Blue-Red diverging
    'sequential': ['#ffffcc', '#a1dab4', '#41b6c4', '#2c7fb8', '#253494'],  # Blue-Green
    'viridis': ['#440154', '#414487', '#2a788e', '#22a884', '#7ad151', '#fde725']  # Viridis
}

def create_multi_feature_choropleth(
    gdf,
    feature_configs,
    center=None,
    zoom_start=6,
    width='100%',
    height='100%'
):
    """
    Create an interactive choropleth map with multiple togglable feature layers using Folium.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        The geodataframe containing geographic and feature data
    feature_configs : list of dict
        List of feature configurations, each containing:
        {
            'name': str (column name in gdf),
            'scale': str ('linear' or 'log'),
            'color_scheme': str (optional, one of 'population', 'political', 'sequential'),
            'legend_name': str (optional, display name for the legend)
        }
    center : tuple, optional
        (lat, lon) center coordinates. If None, will use centroid of the data
    zoom_start : int, default 6
        Initial zoom level
    width : str, default '100%'
        Width of the map in percentage
    height : str, default '100%'
        Height of the map in percentage
        
    Returns:
    --------
    folium.Map
        The interactive map object
    """
    # Input validation
    for config in feature_configs:
        if config['name'] not in gdf.columns:
            raise ValueError(f"Feature '{config['name']}' not found in the GeoDataFrame")
        if config.get('scale') not in ['linear', 'log']:
            raise ValueError(f"Scale for feature '{config['name']}' must be 'linear' or 'log'")
    
    # Calculate center if not provided
    if center is None:
        center = [
            gdf.geometry.centroid.y.mean(),
            gdf.geometry.centroid.x.mean()
        ]
    
    # Create base map
    m = folium.Map(
        location=center,
        zoom_start=zoom_start,
        width=width,
        height=height
    )
    
    # Create feature groups dictionary
    feature_groups = {}
    
    for config in feature_configs:
        feature_name = config['name']
        scale_type = config['scale']
        # Use viridis for log scale, otherwise use specified color scheme
        color_scheme = 'viridis' if scale_type == 'log' else config.get('color_scheme', 'sequential')
        legend_name = config.get('legend_name', feature_name)
        
        # Get appropriate color scheme
        colors = COLOR_SCHEMES.get(color_scheme, COLOR_SCHEMES['sequential'])
        
        # Create feature group
        fg = folium.FeatureGroup(name=legend_name, show=False)
        
        # Handle data scaling and preparation
        data = gdf[feature_name].copy()
        if scale_type == 'log':
            min_val = data.min()
            if min_val <= 0:
                print(f"Warning: {feature_name} contains values ≤ 0. Adding offset for log scale.")
                data = data - min_val + 1
            # Apply log transformation
            data = np.log(data)
        
        # Create colormap
        colormap = LinearColormap(
            colors=colors,
            vmin=data.min(),
            vmax=data.max(),
            caption=legend_name + (' (log scale)' if scale_type == 'log' else '')
        )
        
        # Add choropleth layer
        for idx, row in gdf.iterrows():
            value = row[feature_name]
            if scale_type == 'log':
                if value <= 0:
                    value = value - min_val + 1
                value = np.log(value)
                
            color = colormap(value)
            
            # Format display value
            display_value = row[feature_name]
            if isinstance(display_value, (int, float)):
                if abs(display_value) >= 1000000:
                    display_value = f"{display_value/1000000:.2f}M"
                elif abs(display_value) >= 1000:
                    display_value = f"{display_value/1000:.2f}K"
                elif feature_name.startswith('per_'):  # Percentage values
                    display_value = f"{display_value:.2f}%"
                else:
                    display_value = f"{display_value:.2f}"
            
            # Get state and county names
            state_name = row.get('state_name', row.get('STATE_NAME', ''))
            county_name = row.get('county_name', row.get('COUNTY_NAME', ''))
            
            # Create GeoJSON-style feature
            feature = {
                'type': 'Feature',
                'geometry': row.geometry.__geo_interface__,
                'properties': {
                    'value': display_value,
                    'state': state_name,
                    'county': county_name
                }
            }
            
            # Add polygon to feature group
            folium.GeoJson(
                feature,
                style_function=lambda x, color=color: {
                    'fillColor': color,
                    'color': 'black',
                    'weight': 1,
                    'fillOpacity': 0.7
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=['county', 'state', 'value'],
                    aliases=['County', 'State', legend_name],
                    localize=True,
                    sticky=False,
                    labels=True,
                    style="""
                        background-color: white;
                        border: 2px solid black;
                        border-radius: 3px;
                        box-shadow: 3px 3px 3px rgba(0,0,0,0.2);
                        padding: 5px;
                        font-size: 12px;
                    """
                )
            ).add_to(fg)
        
        # Add colormap to map
        colormap.add_to(m)
        
        # Add feature group to map
        fg.add_to(m)
        
        feature_groups[legend_name] = fg
    
    # Add custom JavaScript for radio button behavior
    layer_control_html = """
    <script type="text/javascript">
    function setupLayerControl() {
        // Get layer control container
        var layerControlContainer = document.querySelector('.leaflet-control-layers-overlays');
        if (!layerControlContainer) {
            setTimeout(setupLayerControl, 100);
            return;
        }

        // Replace checkboxes with radio buttons
        var inputs = layerControlContainer.querySelectorAll('input[type="checkbox"]');
        inputs.forEach(function(input) {
            input.type = 'radio';
            input.name = 'layer-control';
            
            // Update the input's checked state based on layer visibility
            var layer = input._layer;
            if (layer && layer._map) {
                input.checked = layer._map.hasLayer(layer);
            }
            
            // Add change listener to handle layer toggling
            input.addEventListener('change', function(e) {
                if (this.checked) {
                    // Uncheck and hide all other layers
                    inputs.forEach(function(otherInput) {
                        if (otherInput !== input) {
                            otherInput.checked = false;
                            if (otherInput._layer) {
                                otherInput._layer.remove();
                            }
                        }
                    });
                    
                    // Show the selected layer
                    if (this._layer) {
                        this._layer.addTo(this._layer._map);
                    }
                }
            });
        });

        // Monitor layer visibility changes
        var map = document.querySelector('#map');  // Assuming map has id="map"
        if (map && map._leaflet_map) {
            map._leaflet_map.on('layeradd layerremove', function(e) {
                inputs.forEach(function(input) {
                    if (input._layer === e.layer) {
                        input.checked = e.type === 'layeradd';
                    }
                });
            });
        }
    }

    // Initialize when DOM is ready
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', setupLayerControl);
    } else {
        setupLayerControl();
    }

    // Also set up observer to handle dynamic updates
    var observer = new MutationObserver(function(mutations) {
        mutations.forEach(function(mutation) {
            if (mutation.addedNodes.length) {
                setupLayerControl();
            }
        });
    });

    observer.observe(document.body, {
        childList: true,
        subtree: true
    });
    </script>
    """
    
    # Add layer control to map
    folium.LayerControl().add_to(m)
    
    # Add custom JavaScript to handle radio button behavior
    m.get_root().html.add_child(folium.Element(layer_control_html))
    
    return m

In [ ]:
feature_configs = [
    {
        'name': 'INDUSTRY_agriculture_forestry_fishing_and_hunting_and_mining_percent_2020',
        'scale': 'linear',
        'color_scheme': 'political',
        'legend_name': 'Agriculture, Forestry, Fishing, Mining Percent 2020'
    },
    {
        'name': 'per_gop_2020',
        'scale': 'linear',
        'color_scheme': 'political',
        'legend_name': 'GOP Vote Share 2020'
    }
]

map_obj = create_multi_feature_choropleth(
    gdf=census_election_data,
    feature_configs=feature_configs,
    zoom_start=6
)
map_obj

##### Election results all years

In [ ]:
import folium
from folium import plugins
from branca.colormap import LinearColormap
import numpy as np

def create_multiyear_election_map_folium(
    gdf,
    years=[2012, 2016, 2020, 2024],
    center_lat=39.8283,
    center_lon=-98.5795,
    zoom_start=4,
    fill_opacity=0.7,
    default_year=None
):
    """
    Create an interactive election map with mutually exclusive year layers
    controlled by radio buttons.
    """
    if default_year is None:
        default_year = max(years)
    
    required_columns = ['geometry', 'fips', 'county_name', 'state_name']
    for year in years:
        required_columns.extend([f'per_dem_{year}', f'per_gop_{year}'])
    
    gdf = gdf[required_columns].copy()
    
    # Calculate margins and find global scale
    max_diff = float('-inf')
    for year in years:
        dem_col = f'per_dem_{year}'
        gop_col = f'per_gop_{year}'
        margin_col = f'margin_{year}'
        gdf[margin_col] = gdf[gop_col] - gdf[dem_col]
        year_max_diff = max(abs(gdf[margin_col].min()), abs(gdf[margin_col].max()))
        max_diff = max(max_diff, year_max_diff)
    
    max_diff = np.ceil(max_diff * 10) / 10
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles='cartodbpositron'
    )
    
    # Create common colormap
    colormap = LinearColormap(
        colors=[
            '#0000CC',  # Strong Democratic (dark blue)
            '#2A2AFF',  # Democratic
            '#5555FF',  # Lean Democratic
            '#FFFFFF',  # Neutral
            '#FF5555',  # Lean Republican
            '#FF2A2A',  # Republican
            '#CC0000'   # Strong Republican (dark red)
        ],
        vmin=-max_diff,
        vmax=max_diff,
        caption='Party Vote Share Margin (Rep - Dem)'
    )
    
    # Create layers
    for year in years:
        dem_col = f'per_dem_{year}'
        gop_col = f'per_gop_{year}'
        margin_col = f'margin_{year}'
        
        def make_style_function(margin_column):
            def style_function(feature):
                value = feature['properties'][margin_column]
                if value is None:
                    return {'fillColor': '#black', 'fillOpacity': 0, 'weight': 0}
                return {
                    'fillColor': colormap(value),
                    'fillOpacity': fill_opacity,
                    'color': 'none',
                    'weight': 0
                }
            return style_function
        
        def highlight_function(feature):
            return {
                'fillOpacity': fill_opacity,
                'color': 'none',
                'weight': 0
            }
        
        # Create GeoJson layer for this year
        geojson = folium.GeoJson(
            data=gdf.to_crs(epsg='4326').__geo_interface__,
            name=str(year),
            style_function=make_style_function(margin_col),
            highlight_function=highlight_function,
            show=(year == default_year),  # Show only default year
            tooltip=folium.GeoJsonTooltip(
                fields=['county_name', 'state_name', gop_col, dem_col, margin_col],
                aliases=['County', 'State', 
                        f'{year} Republican',
                        f'{year} Democratic',
                        f'{year} Rep-Dem Margin'],
                localize=True,
                sticky=False,
                labels=True,
                style="""
                    background-color: white;
                    border: none;
                    border-radius: 3px;
                    box-shadow: 3px 3px 10px rgba(0,0,0,0.2);
                    font-size: 12px;
                    padding: 8px;
                """
            )
        ).add_to(m)
    
    # Add colormap
    colormap.add_to(m)
    
    # Add layer control with radio buttons
    folium.LayerControl(collapsed=False).add_to(m)
    
    # Add JavaScript to ensure mutual exclusivity and visibility
    m.get_root().html.add_child(folium.Element("""
        <script>
        document.addEventListener('DOMContentLoaded', function() {
            // Get all layer control inputs
            var inputs = document.querySelectorAll('.leaflet-control-layers-selector');
            
            // Initially check the default year
            var defaultChecked = false;
            inputs.forEach(function(input) {
                if (!defaultChecked) {
                    input.checked = true;
                    defaultChecked = true;
                }
            });
            
            // Add click handler to each input
            inputs.forEach(function(input) {
                input.addEventListener('change', function() {
                    if (this.checked) {
                        // Uncheck all other inputs
                        inputs.forEach(function(otherInput) {
                            if (otherInput !== input) {
                                otherInput.checked = false;
                            }
                        });
                    } else {
                        // Ensure at least one layer is always checked
                        var anyChecked = false;
                        inputs.forEach(function(otherInput) {
                            if (otherInput.checked) anyChecked = true;
                        });
                        if (!anyChecked) this.checked = true;
                    }
                });
            });
        });
        </script>
    """))
    
    return m

In [ ]:
m1 = create_multiyear_election_map_folium(
    census_election_data,
    years=[2012, 2016, 2020, 2024],
    default_year=2024  # Optional, defaults to most recent year
)
m1

##### General spatial correlation

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights import Queen, KNN
from esda.moran import Moran, Moran_Local
import warnings
from typing import Union, Tuple, Dict
import mapclassify

def analyze_spatial_correlation(
    gdf: gpd.GeoDataFrame,
    column: str,
    weight_type: str = 'queen',
    k_neighbors: int = 5,
    title: str = None,
    figsize: Tuple[int, int] = (20, 10)
) -> Tuple[Dict, plt.Figure, gpd.GeoDataFrame]:
    """
    Analyzes and visualizes spatial correlation for any column in a GeoDataFrame.
    
    Parameters:
    -----------
    gdf : gpd.GeoDataFrame
        GeoDataFrame containing geometry and data
    column : str
        Name of column to analyze
    weight_type : str, optional (default='queen')
        Type of spatial weights to use ('queen' or 'knn')
    k_neighbors : int, optional (default=5)
        Number of neighbors for KNN weights
    title : str, optional
        Custom title for plots
    figsize : tuple, optional (default=(20, 10))
        Figure size for plots
    """
    # Input validation
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Input must be a GeoDataFrame")
    if column not in gdf.columns:
        raise ValueError(f"Column '{column}' not found in GeoDataFrame")
    
    # Create copy to avoid modifying original
    gdf_analysis = gdf.copy()
    
    # Handle missing values
    if gdf_analysis[column].isnull().any():
        print(f"Warning: {gdf_analysis[column].isnull().sum()} missing values found and will be removed")
        gdf_analysis = gdf_analysis.dropna(subset=[column])
    
    # Create spatial weights matrix
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if weight_type.lower() == 'queen':
            weights = Queen.from_dataframe(gdf_analysis, use_index=True)
        elif weight_type.lower() == 'knn':
            weights = KNN.from_dataframe(gdf_analysis, k=k_neighbors)
        else:
            raise ValueError("weight_type must be either 'queen' or 'knn'")
    
    # Handle islands if using queen weights
    if weight_type.lower() == 'queen':
        islands = weights.islands
        if len(islands) > 0:
            print(f"Removing {len(islands)} isolated areas from analysis")
            gdf_analysis = gdf_analysis[~gdf_analysis.index.isin(islands)].copy()
            weights = Queen.from_dataframe(gdf_analysis, use_index=True)
    
    weights.transform = 'r'  # Row-standardize weights
    
    # Calculate Global Moran's I
    moran = Moran(gdf_analysis[column], weights)
    
    # Calculate Local Moran's I
    local_moran = Moran_Local(gdf_analysis[column], weights)
    
    # Add local indicators to dataframe
    gdf_analysis['local_moran_i'] = local_moran.Is
    gdf_analysis['local_moran_p'] = local_moran.p_sim
    
    # Classify clusters
    gdf_analysis['cluster_type'] = 'Not Significant'
    sig_mask = local_moran.p_sim < 0.05
    
    # Standardize values for clustering
    std_val = (gdf_analysis[column] - gdf_analysis[column].mean()) / gdf_analysis[column].std()
    lag_val = weights.sparse.dot(std_val)
    
    # Assign cluster types
    gdf_analysis.loc[sig_mask & (std_val > 0) & (lag_val > 0), 'cluster_type'] = 'High-High'
    gdf_analysis.loc[sig_mask & (std_val < 0) & (lag_val < 0), 'cluster_type'] = 'Low-Low'
    gdf_analysis.loc[sig_mask & (std_val > 0) & (lag_val < 0), 'cluster_type'] = 'High-Low'
    gdf_analysis.loc[sig_mask & (std_val < 0) & (lag_val > 0), 'cluster_type'] = 'Low-High'
    
    # Create visualizations
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Plot 1: Original Values
    plot1 = gdf_analysis.plot(
        column=column,
        ax=axes[0],
        legend=True,
        cmap='viridis'
    )
    axes[0].set_title(f"Distribution of {column}")
    axes[0].axis('off')
    
    # Add colorbar with label
    cbar1 = plt.colorbar(plot1.get_children()[0], ax=axes[0])
    cbar1.set_label(column, rotation=270, labelpad=15)
    
    # Plot 2: Local Moran's I Values
    plot2 = gdf_analysis.plot(
        column='local_moran_i',
        ax=axes[1],
        legend=True,
        cmap='RdBu'
    )
    axes[1].set_title("Local Moran's I Values")
    axes[1].axis('off')
    
    # Add colorbar with label
    cbar2 = plt.colorbar(plot2.get_children()[0], ax=axes[1])
    cbar2.set_label("Local Moran's I", rotation=270, labelpad=15)
    
    # Plot 3: Cluster Types with categorical legend
    cluster_colors = {'High-High': '#d7191c', 
                     'Low-Low': '#2c7bb6', 
                     'High-Low': '#fdae61', 
                     'Low-High': '#abd9e9', 
                     'Not Significant': '#ffffbf'}
    
    for cluster_type, color in cluster_colors.items():
        mask = gdf_analysis['cluster_type'] == cluster_type
        if mask.any():
            gdf_analysis[mask].plot(
                color=color,
                ax=axes[2],
                label=cluster_type
            )
    
    axes[2].set_title('Spatial Clusters')
    axes[2].axis('off')
    axes[2].legend(title='Cluster Type', bbox_to_anchor=(1.3, 1))
    
    if title:
        fig.suptitle(title, fontsize=16, y=1.05)
    
    plt.tight_layout()
    
    # Calculate summary statistics
    results = {
        'global_statistics': {
            'morans_i': moran.I,
            'p_value': moran.p_sim,
            'z_score': moran.z_sim
        },
        'cluster_summary': gdf_analysis['cluster_type'].value_counts().to_dict(),
        'local_statistics': {
            'mean_local_i': gdf_analysis['local_moran_i'].mean(),
            'significant_clusters': (gdf_analysis['local_moran_p'] < 0.05).sum(),
            'percent_significant': (gdf_analysis['local_moran_p'] < 0.05).mean() * 100
        }
    }
    
    return results, fig, gdf_analysis

def print_spatial_correlation_results(results: Dict, column_name: str = "selected variable"):
    """
    Prints formatted spatial correlation analysis results.
    """
    print(f"Spatial Correlation Analysis Results for {column_name}\n")
    
    print("Global Moran's I Statistics:")
    print(f"Moran's I: {results['global_statistics']['morans_i']:.3f}")
    print(f"P-value: {results['global_statistics']['p_value']:.3f}")
    print(f"Z-score: {results['global_statistics']['z_score']:.3f}")
    
    print("\nLocal Statistics:")
    print(f"Mean Local Moran's I: {results['local_statistics']['mean_local_i']:.3f}")
    print(f"Number of Significant Clusters: {results['local_statistics']['significant_clusters']}")
    print(f"Percent Significant: {results['local_statistics']['percent_significant']:.1f}%")
    
    print("\nCluster Type Distribution:")
    for cluster_type, count in results['cluster_summary'].items():
        print(f"{cluster_type}: {count} areas")

In [ ]:
results, fig, gdf_analyzed = analyze_spatial_correlation(
    gdf=census_election_data,
    column='religion_evangelical_lutheran_church_in_america_percent_adherents_of_total_adherents',
    title='Evangelical Lutheran Church Adherents Analysis'
)

print_spatial_correlation_results(results)
plt.show()

##### Enhanced Spatial correlation map (folium)

In [ ]:
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd
import numpy as np
from libpysal.weights import Queen, KNN
from esda.moran import Moran, Moran_Local
import warnings
from typing import Union, Tuple, Dict
from branca.colormap import LinearColormap

def analyze_spatial_correlation_interactive(
    gdf: gpd.GeoDataFrame,
    column: str,
    weight_type: str = 'queen',
    k_neighbors: int = 5,
    center: Tuple[float, float] = None,
    zoom_start: int = 4
) -> Tuple[Dict, folium.Map]:
    """
    Creates an interactive Folium map showing spatial correlation analysis with three layers:
    1. Original value distribution
    2. Local Moran's I values
    3. Cluster types
    
    Parameters:
    -----------
    gdf : gpd.GeoDataFrame
        GeoDataFrame containing geometry and data
    column : str
        Name of column to analyze
    weight_type : str, optional (default='queen')
        Type of spatial weights to use ('queen' or 'knn')
    k_neighbors : int, optional (default=5)
        Number of neighbors for KNN weights
    center : tuple, optional
        (lat, lon) center coordinates for the map
    zoom_start : int, optional (default=4)
        Initial zoom level for the map
    """
    # Input validation and data preparation
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Input must be a GeoDataFrame")
    if column not in gdf.columns:
        raise ValueError(f"Column '{column}' not found in GeoDataFrame")
    
    # Create copy to avoid modifying original
    gdf_analysis = gdf.copy()
    
    # Handle missing values
    if gdf_analysis[column].isnull().any():
        print(f"Warning: {gdf_analysis[column].isnull().sum()} missing values found and will be removed")
        gdf_analysis = gdf_analysis.dropna(subset=[column])
    
    # Create spatial weights matrix
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if weight_type.lower() == 'queen':
            weights = Queen.from_dataframe(gdf_analysis, use_index=True)
        elif weight_type.lower() == 'knn':
            weights = KNN.from_dataframe(gdf_analysis, k=k_neighbors)
        else:
            raise ValueError("weight_type must be either 'queen' or 'knn'")
    
    # Handle islands if using queen weights
    if weight_type.lower() == 'queen':
        islands = weights.islands
        if len(islands) > 0:
            print(f"Removing {len(islands)} isolated areas from analysis")
            gdf_analysis = gdf_analysis[~gdf_analysis.index.isin(islands)].copy()
            weights = Queen.from_dataframe(gdf_analysis, use_index=True)
    
    weights.transform = 'r'  # Row-standardize weights
    
    # Calculate Global Moran's I
    moran = Moran(gdf_analysis[column], weights)
    
    # Calculate Local Moran's I
    local_moran = Moran_Local(gdf_analysis[column], weights)
    
    # Add local indicators to dataframe
    gdf_analysis['local_moran_i'] = local_moran.Is
    gdf_analysis['local_moran_p'] = local_moran.p_sim
    
    # Classify clusters
    gdf_analysis['cluster_type'] = 'Not Significant'
    sig_mask = local_moran.p_sim < 0.05
    
    # Standardize values for clustering
    std_val = (gdf_analysis[column] - gdf_analysis[column].mean()) / gdf_analysis[column].std()
    lag_val = weights.sparse.dot(std_val)
    
    # Assign cluster types
    gdf_analysis.loc[sig_mask & (std_val > 0) & (lag_val > 0), 'cluster_type'] = 'High-High'
    gdf_analysis.loc[sig_mask & (std_val < 0) & (lag_val < 0), 'cluster_type'] = 'Low-Low'
    gdf_analysis.loc[sig_mask & (std_val > 0) & (lag_val < 0), 'cluster_type'] = 'High-Low'
    gdf_analysis.loc[sig_mask & (std_val < 0) & (lag_val > 0), 'cluster_type'] = 'Low-High'
    
    # Calculate map center if not provided
    if center is None:
        center = [
            gdf_analysis.geometry.centroid.y.mean(),
            gdf_analysis.geometry.centroid.x.mean()
        ]
    
    # Create base map
    m = folium.Map(location=center, zoom_start=zoom_start)
    
    # Create feature groups for each layer
    fg_original = folium.FeatureGroup(name=f"Distribution of {column}")
    fg_moran = folium.FeatureGroup(name="Local Moran's I Values")
    fg_clusters = folium.FeatureGroup(name="Spatial Clusters")
    
    # Create colormaps
    colormap_original = LinearColormap(
        colors=['#440154', '#414487', '#2a788e', '#22a884', '#7ad151', '#fde725'],
        vmin=gdf_analysis[column].min(),
        vmax=gdf_analysis[column].max(),
        caption=f"Distribution of {column}"
    )
    
    colormap_moran = LinearColormap(
        colors=['#ca0020', '#f4a582', '#f7f7f7', '#92c5de', '#0571b0'],
        vmin=gdf_analysis['local_moran_i'].min(),
        vmax=gdf_analysis['local_moran_i'].max(),
        caption="Local Moran's I Values"
    )
    
    cluster_colors = {
        'High-High': '#d7191c',
        'Low-Low': '#2c7bb6',
        'High-Low': '#fdae61',
        'Low-High': '#abd9e9',
        'Not Significant': '#ffffbf'
    }
    
    # Add layers
    # 1. Original Distribution Layer
    for idx, row in gdf_analysis.iterrows():
        color = colormap_original(row[column])
        folium.GeoJson(
            row.geometry.__geo_interface__,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            },
            tooltip=f"{column}: {row[column]:.2f}"
        ).add_to(fg_original)
    
    # 2. Local Moran's I Layer
    for idx, row in gdf_analysis.iterrows():
        color = colormap_moran(row['local_moran_i'])
        folium.GeoJson(
            row.geometry.__geo_interface__,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            },
            tooltip=f"Local Moran's I: {row['local_moran_i']:.3f}<br>P-value: {row['local_moran_p']:.3f}"
        ).add_to(fg_moran)
    
    # 3. Cluster Types Layer
    for idx, row in gdf_analysis.iterrows():
        color = cluster_colors[row['cluster_type']]
        folium.GeoJson(
            row.geometry.__geo_interface__,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            },
            tooltip=f"Cluster Type: {row['cluster_type']}"
        ).add_to(fg_clusters)
    
    # Add feature groups to map
    fg_original.add_to(m)
    fg_moran.add_to(m)
    fg_clusters.add_to(m)
    
    # Add colormaps to map
    colormap_original.add_to(m)
    colormap_moran.add_to(m)
    
    # Add cluster type legend
    legend_html = """
    <div style="position: fixed; bottom: 50px; right: 50px; z-index: 1000; background-color: white; 
                padding: 10px; border: 2px solid grey; border-radius: 5px">
    <p><strong>Cluster Types</strong></p>
    """
    for cluster_type, color in cluster_colors.items():
        legend_html += f"""
        <p><i class="fa fa-square fa-1x" style="color:{color}"></i> {cluster_type}</p>
        """
    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Calculate summary statistics
    results = {
        'global_statistics': {
            'morans_i': moran.I,
            'p_value': moran.p_sim,
            'z_score': moran.z_sim
        },
        'cluster_summary': gdf_analysis['cluster_type'].value_counts().to_dict(),
        'local_statistics': {
            'mean_local_i': gdf_analysis['local_moran_i'].mean(),
            'significant_clusters': (gdf_analysis['local_moran_p'] < 0.05).sum(),
            'percent_significant': (gdf_analysis['local_moran_p'] < 0.05).mean() * 100
        }
    }
    
    return results, m

In [ ]:
# Example usage
results, interactive_map = analyze_spatial_correlation_interactive(
    gdf=census_election_data,
    column='religion_american_baptist_churches_in_the_usa_percent_adherents_of_total_adherents'
)

# Display results
print_spatial_correlation_results(results)

# Display map
interactive_map

###### Get all spatial correlations

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from libpysal.weights import Queen, KNN
from esda.moran import Moran, Moran_Local
import warnings
from typing import Union, Tuple, Dict, List
from dataclasses import dataclass
from collections import defaultdict

@dataclass
class SpatialCorrelationResult:
    """Data class to store correlation results for a single feature"""
    feature: str
    global_morans_i: float
    global_p_value: float
    global_z_score: float
    mean_local_i: float
    significant_clusters: int
    percent_significant: float
    cluster_counts: Dict[str, int]

def calculate_multi_feature_correlation(
    gdf: gpd.GeoDataFrame,
    features: List[str] = None,
    weight_type: str = 'queen',
    k_neighbors: int = 5,
    exclude_columns: List[str] = None
) -> List[SpatialCorrelationResult]:
    """
    Calculates spatial correlation metrics for multiple features in a GeoDataFrame.
    
    Parameters:
    -----------
    gdf : gpd.GeoDataFrame
        GeoDataFrame containing geometry and data
    features : List[str], optional
        List of column names to analyze. If None, will analyze all numeric columns
    weight_type : str, optional (default='queen')
        Type of spatial weights to use ('queen' or 'knn')
    k_neighbors : int, optional (default=5)
        Number of neighbors for KNN weights
    exclude_columns : List[str], optional
        List of column names to exclude from analysis
        
    Returns:
    --------
    List[SpatialCorrelationResult]
        List of correlation results for each feature
    """
    # Input validation
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Input must be a GeoDataFrame")
        
    # If no features specified, use all numeric columns
    if features is None:
        features = gdf.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove excluded columns
    if exclude_columns:
        features = [f for f in features if f not in exclude_columns]
    
    results = []
    
    # Create spatial weights matrix once
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if weight_type.lower() == 'queen':
            weights = Queen.from_dataframe(gdf, use_index=True)
            # Handle islands
            islands = weights.islands
            if len(islands) > 0:
                gdf = gdf[~gdf.index.isin(islands)].copy()
                weights = Queen.from_dataframe(gdf, use_index=True)
        elif weight_type.lower() == 'knn':
            weights = KNN.from_dataframe(gdf, k=k_neighbors)
        else:
            raise ValueError("weight_type must be either 'queen' or 'knn'")
    
    weights.transform = 'r'  # Row-standardize weights
    
    # Analyze each feature
    for feature in features:
        # Skip if feature has all missing values
        if gdf[feature].isnull().all():
            continue
            
        # Create copy of data without missing values for this feature
        gdf_clean = gdf.dropna(subset=[feature]).copy()
        
        if len(gdf_clean) < 2:  # Skip if not enough data
            continue
            
        try:
            # Calculate Global Moran's I
            moran = Moran(gdf_clean[feature], weights)
            
            # Calculate Local Moran's I
            local_moran = Moran_Local(gdf_clean[feature], weights)
            
            # Calculate cluster types
            sig_mask = local_moran.p_sim < 0.05
            std_val = (gdf_clean[feature] - gdf_clean[feature].mean()) / gdf_clean[feature].std()
            lag_val = weights.sparse.dot(std_val)
            
            cluster_types = np.full(len(gdf_clean), 'Not Significant', dtype=object)
            cluster_types[sig_mask & (std_val > 0) & (lag_val > 0)] = 'High-High'
            cluster_types[sig_mask & (std_val < 0) & (lag_val < 0)] = 'Low-Low'
            cluster_types[sig_mask & (std_val > 0) & (lag_val < 0)] = 'High-Low'
            cluster_types[sig_mask & (std_val < 0) & (lag_val > 0)] = 'Low-High'
            
            cluster_counts = dict(pd.Series(cluster_types).value_counts())
            
            result = SpatialCorrelationResult(
                feature=feature,
                global_morans_i=moran.I,
                global_p_value=moran.p_sim,
                global_z_score=moran.z_sim,
                mean_local_i=np.mean(local_moran.Is),
                significant_clusters=np.sum(sig_mask),
                percent_significant=np.mean(sig_mask) * 100,
                cluster_counts=cluster_counts
            )
            
            results.append(result)
            
        except Exception as e:
            print(f"Error analyzing feature {feature}: {str(e)}")
            continue
            
    return results

def get_top_correlations(
    results: List[SpatialCorrelationResult],
    n: int = 5,
    p_threshold: float = 0.05,
    sort_by: str = 'global'
) -> pd.DataFrame:
    """
    Returns the top n features by spatial correlation.
    
    Parameters:
    -----------
    results : List[SpatialCorrelationResult]
        List of correlation results
    n : int, optional (default=5)
        Number of top features to return
    p_threshold : float, optional (default=0.05)
        Only include results with p-value below this threshold
    sort_by : str, optional (default='global')
        Sort by 'global' or 'local' Moran's I
        
    Returns:
    --------
    pd.DataFrame
        Sorted DataFrame of top correlations
    """
    # Convert results to DataFrame
    df = pd.DataFrame([
        {
            'feature': r.feature,
            'global_morans_i': r.global_morans_i,
            'global_p_value': r.global_p_value,
            'global_z_score': r.global_z_score,
            'mean_local_i': r.mean_local_i,
            'significant_clusters': r.significant_clusters,
            'percent_significant': r.percent_significant
        }
        for r in results
    ])
    
    # Filter by p-value
    df = df[df['global_p_value'] < p_threshold]
    
    # Sort by specified metric
    sort_col = 'global_morans_i' if sort_by == 'global' else 'mean_local_i'
    df = df.sort_values(by=sort_col, ascending=False)
    
    return df.head(n)

In [ ]:
# # Calculate correlations for all numeric features
# results = calculate_multi_feature_correlation(
#     census_election_data,
#     weight_type='queen',  # or 'knn'
#     k_neighbors=5  # only needed if using 'knn'
# )

# Get top n features by global Moran's I
top_global = get_top_correlations(
    results,
    n=1000,
    p_threshold=0.05,
    sort_by='global'
)

# Get top n features by local Moran's I
top_local = get_top_correlations(
    results,
    n=1000,
    p_threshold=0.05,
    sort_by='local'
)

In [ ]:
top_global

In [ ]:
# results
top_global.sort_values(by='percent_significant', ascending=False).head(100)
# top_local.sort_values(by='mean_local_i', ascending=False)

#### Scratch

In [ ]:
census_election_data.state_name.str.lower().unique()

In [ ]:
import pandas as pd
import unidecode

def clean_place_name(name, state=None):
    """
    Clean place names for consistent matching.
    Now handles state-specific naming conventions.
    
    Parameters:
    name: str, place name to clean
    state: str, optional state abbreviation to apply state-specific rules
    """
    if pd.isna(name):
        return name
    
    # Convert to string and clean accented characters
    name = str(name).strip()
    name = unidecode.unidecode(name)
    
    # Remove common suffixes based on state
    suffixes_to_remove = {
        'LA': [' County', ' Parish'],
        'AK': [' County', ' Borough', ' Census Area', ' Municipality'],
        'VA': [' County', ' city'],  # Virginia has independent cities
        'DC': [' County'],  # Handle DC separately
        None: [' County', ' Parish', ' Borough', ' Census Area', ' Municipality', ' city']
    }
    
    # Get the appropriate list of suffixes to remove
    suffixes = suffixes_to_remove.get(state, suffixes_to_remove[None])
    
    # Remove any matching suffix
    for suffix in suffixes:
        if name.endswith(suffix):
            name = name[:-len(suffix)]
            break
    
    # Standardize specific terms
    replacements = {
        'Municipality': 'Municipio',
        'City and Borough': 'Borough',
        'Census Area': 'Census Area',
        ' city': ' City',
        'LaSalle': 'La Salle',
        'LaPorte': 'La Porte',
        'DeSoto': 'De Soto',
        'DeKalb': 'De Kalb',
        'St.': 'St',
        'Ste.': 'Ste'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    # Add back the appropriate suffix based on state
    if state == 'LA':
        name = name + ' Parish'
    elif state == 'AK':
        # Need to check original name to determine correct suffix
        if ' Borough' in str(name):
            name = name + ' Borough'
        elif ' Census Area' in str(name):
            name = name + ' Census Area'
        else:
            name = name  # No suffix for municipalities
    
    return name.strip()

def add_fips_codes_to_election_data(election_df, fips_csv_path):
    """
    Add FIPS codes to election data by matching county and state.
    
    Parameters:
    election_df (pandas.DataFrame): Election dataframe with State and County columns
    fips_csv_path (str): Path to the FIPS CSV file
    
    Returns:
    pandas.DataFrame: Election dataframe with FIPS codes added
    """
    # Read FIPS data
    fips_df = pd.read_csv(fips_csv_path)
    
    # Filter out non-place rows
    fips_df = fips_df[~fips_df['name'].isna()]
    fips_df = fips_df[fips_df['name'].str.contains('County|Borough|Parish|Census Area|city|Municipio', 
                                                  case=False, na=False)]
    
    # Create state abbreviation mapping
    state_abbr = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
    state_map = dict(zip(state_abbr['State'], state_abbr['Abbreviation']))
    
    # Add state abbreviations to election data
    election_df = election_df.copy()
    election_df['state_abbr'] = election_df['State'].map(state_map)
    
    # Clean up names in both dataframes, using state information
    election_df['clean_county'] = election_df.apply(
        lambda x: clean_place_name(x['County'], x['state_abbr']), axis=1
    )
    fips_df['clean_name'] = fips_df.apply(
        lambda x: clean_place_name(x['name'], x['state']), axis=1
    )
    
    # Merge the dataframes
    result = pd.merge(
        election_df,
        fips_df[['fips', 'clean_name', 'state']],
        how='left',
        left_on=['clean_county', 'state_abbr'],
        right_on=['clean_name', 'state']
    )
    
    # Drop working columns
    result = result.drop(['clean_county', 'clean_name', 'state'], axis=1)
    
    # Convert FIPS codes to strings with leading zeros
    result['fips'] = result['fips'].astype(str).str.zfill(5)
    
    # Check for unmatched counties
    unmatched = result[result['fips'].isna()]
    if len(unmatched) > 0:
        print(f"\nWarning: {len(unmatched)} counties could not be matched with FIPS codes:")
        for _, row in unmatched.iterrows():
            print(f"- {row['County']}, {row['State']}")
    
    # Reorder columns to put FIPS code after State
    cols = list(result.columns)
    state_idx = cols.index('State')
    cols.remove('fips')
    cols.insert(state_idx + 1, 'fips')
    result = result[cols]
    
    return result

In [ ]:
# read the 2024 results csv, may have weird encoding so use latin1
# results_2024 = pd.read_csv('../../data/election/2024_county_results_general.csv')
results_2024 = pd.read_csv('../../data/election/2024_county_results_general.csv', encoding='latin1')

In [ ]:
results_2024

In [ ]:
# drop rows that contain a the text 'Est' in the County column
results_2024 = results_2024[~results_2024['County'].str.contains('Est')]
# drop cols with title Robert F. Kennedy Jr	Chase Oliver	Jill Stein
results_2024 = results_2024.drop(columns=['Robert F. Kennedy Jr', 'Chase Oliver', 'Jill Stein'])

In [ ]:
# drop all rows that have '-' in any column
results_2024 = results_2024[~results_2024.isin(['-']).any(axis=1)]

In [ ]:
# change the percent columns to floats (they are currently strings with percent signs) (the cols are Kamala Harris and Donald Trump)
percent_columns = ['Kamala Harris', 'Donald Trump']
results_2024[percent_columns] = results_2024[percent_columns].replace('%', '', regex=True).astype(float)/100

In [ ]:
# add ' County' to the end of every value in the County column
results_2024['County'] = results_2024['County'] + ' County'

In [ ]:
# Assuming your election dataframe is called 'election_df'
fips_path = '../../data/election/fips_state_and_county.csv'
votes_2024_fips = add_fips_codes_to_election_data(results_2024, fips_path)

In [ ]:
# change the titles of Kamala Harris,	Donald Trump to 'per_dem_2024', 'per_gop_2024'
votes_2024_fips = votes_2024_fips.rename(columns={'Kamala Harris': 'per_dem_2024', 'Donald Trump': 'per_gop_2024'})

In [ ]:
pd.set_option('display.max_rows', 300)

In [ ]:
# votes_2024_fips
# drop all rows where fips is '00nan'
votes_2024_fips_fixed = votes_2024_fips[votes_2024_fips['fips'] != '00nan']

In [ ]:
# set fips to float
votes_2024_fips_fixed['fips'] = votes_2024_fips_fixed['fips'].astype(float)

In [ ]:
votes_2024_fips_fixed.fips.dtype

In [ ]:
# census_election_data.fips
# merge the votes_2024_fips dataframe with the census_election_data dataframe on the fips column
merged_data = pd.merge(census_election_data, votes_2024_fips_fixed, on='fips', how='left')
merged_data

In [ ]:
# drop per_dem_2024	per_gop_2024	president_winner_2024	flipped_2016	flipped_2020	flipped_2024	winner_2024	winner_2024_numeric	flipped_2012_2016	flipped_2016_2020	flipped_2020_2024	flipped_dem_2016	flipped_gop_2016	flipped_dem_2020	flipped_gop_2020	flipped_dem_2024	flipped_gop_2024 from census_election_data
census_election_data = census_election_data.drop(columns=['per_dem_2024', 'per_gop_2024', 'president_winner_2024', 'flipped_2016', 'flipped_2020', 'flipped_2024', 'winner_2024', 'winner_2024_numeric', 'flipped_2012_2016', 'flipped_2016_2020', 'flipped_2020_2024', 'flipped_dem_2016', 'flipped_gop_2016', 'flipped_dem_2020', 'flipped_gop_2020', 'flipped_dem_2024', 'flipped_gop_2024'])

In [ ]:
# rename state_abbr_x to state_abbr, drop state_abbr_y, State, County
merged_data = merged_data.rename(columns={'state_abbr_x': 'state_abbr'})
merged_data = merged_data.drop(columns=['state_abbr_y', 'State', 'County'])

In [ ]:
# in merged_data, add a column called winner_2024_numeric that is 1 if per_gop_2024 > per_dem_2024, 0 otherwise
merged_data['winner_2024_numeric'] = np.where(merged_data['per_gop_2024'] > merged_data['per_dem_2024'], 1, 0)
# in merged_data, add a column called president_winner_2024 that is 'GOP'
merged_data['president_winner_2024'] = 'GOP'
# in merged_data, add a column called winner_2024 that is 'GOP' if winner_2024_numeric is 1, 'DEM' otherwise
merged_data['winner_2024'] = np.where(merged_data['winner_2024_numeric'] == 1, 'GOP', 'DEM')

In [ ]:
merged_data.head()

In [ ]:
# in merged_data, add a column called state_electoral_votes that is the number of electoral votes for the state
state_electoral_votes = {
    'AL': 9,
    'AK': 3,
    'AZ': 11,
    'AR': 6,
    'CA': 55,
    'CO': 9,
    'CT': 7,
    'DE': 3,
    'DC': 3,
    'FL': 29,
    'GA': 16,
    'HI': 4,
    'ID': 4,
    'IL': 20,
    'IN': 11,
    'IA': 6,
    'KS': 6,
    'KY': 8,
    'LA': 8,
    'ME': 4,
    'MD': 10,
    'MA': 11,
    'MI': 16,
    'MN': 10,
    'MS': 6,
    'MO': 10,
    'MT': 3,
    'NE': 5,
    'NV': 6,
    'NH': 4,
    'NJ': 14,
    'NM': 5,
    'NY': 29,
    'NC': 15,
    'ND': 3,
    'OH': 18,
    'OK': 7,
    'OR': 7,
    'PA': 20,
    'RI': 4,
    'SC': 9,
    'SD': 3,
    'TN': 11,
    'TX': 38,
    'UT': 6,
    'VT': 3,
    'VA': 13,
    'WA': 12,
    'WV': 5,
    'WI': 10,
    'WY': 3
}

merged_data['state_electoral_votes'] = merged_data['state_abbr'].map(state_electoral_votes)

In [ ]:
# in merged_data, create a column called state_pop_approx that is the total population of the state.  use the Total:_2020 column as the county population, and sum these values for each state
state_pop_approx = merged_data.groupby('state_abbr')['Total:_2020'].sum()
merged_data['state_pop_approx'] = merged_data['state_abbr'].map(state_pop_approx)

In [ ]:
# in merged_data, create a column called pct_pop_state that is the Total:_2020 divided by the sum of state_pop_approx
merged_data['pct_pop_state'] = merged_data['Total:_2020'] / merged_data['state_pop_approx']

In [ ]:
# show the top 10 rows where pct_pop_state is highest
merged_data.sort_values('pct_pop_state', ascending=False).head(10)

In [ ]:
# in merged data, create a double histogram where the left side is the distribution of pct_pop_state where Metro_2013 is 0, and the right side is the distribution of pct_pop_state where Metro_2013 is 1
# plot them on the same axis so they can be easily compared

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Filter data for each histogram
metro_0 = merged_data[merged_data['Metro_2013'] == 0]
metro_1 = merged_data[merged_data['Metro_2013'] == 1]

# Plot histograms
sns.histplot(metro_0['pct_pop_state'], bins=20, color='skyblue', label='Non-Metro', ax=ax)
sns.histplot(metro_1['pct_pop_state'], bins=20, color='salmon', label='Metro', ax=ax)

# Add labels and title
plt.xlabel('Population Percentage')
plt.ylabel('Count')
plt.title('Distribution of Population Percentage by Metro Status')
plt.legend(title='Metro Status')

# Show plot
plt.show()

In [ ]:
# in merged_data, show col names that contain 'president'
president_columns = [col for col in merged_data.columns if 'flipped' in col]
president_columns

In [ ]:
merged_data = merged_data.drop(columns=president_columns)

In [ ]:
def calculate_flips(merged_data):
    """
    Calculate party flips between election cycles.
    3 = DEM to GOP flip (0 to 1)
    2 = GOP to DEM flip (1 to 0)
    0 = No flip (stayed same party)
    """
    # 2012 to 2016 flips
    merged_data['flipped_2012_2016'] = 0  # Initialize with no flip
    dem_to_gop_mask = (merged_data['winner_2012_numeric'] == 0) & (merged_data['winner_2016_numeric'] == 1)
    gop_to_dem_mask = (merged_data['winner_2012_numeric'] == 1) & (merged_data['winner_2016_numeric'] == 0)
    merged_data.loc[dem_to_gop_mask, 'flipped_2012_2016'] = 3
    merged_data.loc[gop_to_dem_mask, 'flipped_2012_2016'] = 2
    
    # 2016 to 2020 flips
    merged_data['flipped_2016_2020'] = 0  # Initialize with no flip
    dem_to_gop_mask = (merged_data['winner_2016_numeric'] == 0) & (merged_data['winner_2020_numeric'] == 1)
    gop_to_dem_mask = (merged_data['winner_2016_numeric'] == 1) & (merged_data['winner_2020_numeric'] == 0)
    merged_data.loc[dem_to_gop_mask, 'flipped_2016_2020'] = 3
    merged_data.loc[gop_to_dem_mask, 'flipped_2016_2020'] = 2
    
    # 2020 to 2024 flips
    merged_data['flipped_2020_2024'] = 0  # Initialize with no flip
    dem_to_gop_mask = (merged_data['winner_2020_numeric'] == 0) & (merged_data['winner_2024_numeric'] == 1)
    gop_to_dem_mask = (merged_data['winner_2020_numeric'] == 1) & (merged_data['winner_2024_numeric'] == 0)
    merged_data.loc[dem_to_gop_mask, 'flipped_2020_2024'] = 3
    merged_data.loc[gop_to_dem_mask, 'flipped_2020_2024'] = 2
    
    return merged_data

In [ ]:
merged_data = calculate_flips(merged_data)

In [ ]:
# Check counts of each type of flip for each period
for col in ['flipped_2012_2016', 'flipped_2016_2020', 'flipped_2020_2024']:
    print(f"\n{col} value counts:")
    print(merged_data[col].value_counts())

In [ ]:
# save merged_data to geojson ('../../data/election/final_data/county_demographics_with_elections_2012_2024.geojson')
merged_data.to_file('../../data/election/final_data/county_demographics_with_elections_2012_2024.geojson', driver='GeoJSON')